In [1]:
%%markdown
# OpenTelemetry for Developers, Data Engineers and Data Scientists

> Welcome to our quick guide to [OpenTelemetry](https://opentelemetry.io/) for Developers, Data Engineers, Data Scientists and just about everyone who prefers to learn by looking at actual data.
> 
> In the following sections we'll introduce the various OpenTelemetry signals - `logs`, `metrics`, `traces` and the upcoming `profiles` as well as additional concepts like `resources` and `scopes`.
> Samples shown are based on actual OpenTelemetry data emitted by a [demo Astronomy webshop app](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) and collected by [an OTLP Parquet collector](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) developed by [mishmash.io](https://mishmash.io/).
>
> At [mishmash.io](https://mishmash.io/) we use OpenTelemetry as a development tool to help us:
> - develop faster and more efficient distributed database
> - ensure the quality of each release
> - apply deep analytics to understand better the complexities of distributed computing
>
> To find out more visit our [Open Source Development Tools page.](https://mishmash.io/open_source)

# OpenTelemetry for Developers, Data Engineers and Data Scientists

> Welcome to our quick guide to [OpenTelemetry](https://opentelemetry.io/) for Developers, Data Engineers, Data Scientists and just about everyone who prefers to learn by looking at actual data.
> 
> In the following sections we'll introduce the various OpenTelemetry signals - `logs`, `metrics`, `traces` and the upcoming `profiles` as well as additional concepts like `resources` and `scopes`.
> Samples shown are based on actual OpenTelemetry data emitted by a [demo Astronomy webshop app](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) and collected by [an OTLP Parquet collector](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) developed by [mishmash.io](https://mishmash.io/).
>
> At [mishmash.io](https://mishmash.io/) we use OpenTelemetry as a development tool to help us:
> - develop faster and more efficient distributed database
> - ensure the quality of each release
> - apply deep analytics to understand better the complexities of distributed computing
>
> To find out more visit our [Open Source Development Tools page.](https://mishmash.io/open_source)


In [2]:
# First, we'll need some imports
# To illustrate we'll use Pandas DataFrames
import pandas as pd
# PyArrow to read Apache Parquet files
import pyarrow as pa
# ...and NumPy
import numpy as np

In [3]:
# As the example DataFrames might contain a lot of columns and rows, let's configure a more appropriate way of displaying them:
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_rows', 200)
pd.options.display.max_colwidth = 100


In [4]:
%%markdown

---

# Logs signal

Let's begin with the [OpenTelemetry logs signal.](https://opentelemetry.io/docs/specs/otel/logs/)

As the name suggests it contains individual log messages generated by your code and the libraries you're using. Includes timestamps (in the 
`time_unix_nano` and `observed_time_unix_nano` columns), severity level [INFO, WARNING, ERROR...] (in the `severity_number` and `severity_text` columns),
the log message itself (in the `body_*` columns) and additional `attributes` (more about attributes below).

A notable OpenTelemetry extension over traditional logging is the ability to correlate log messages to the traces (`trace_id` column) and spans (`span_id` column) within which they were generated. (`traces` are intoduced later in this notebook.)

Let's jump to exploring some raw `logs.`

> ***Note:*** Raw data displayed below is ***as recorded by*** our [Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet), which adds a few extra columns:
> - `batch_timestamp`
> - `batch_UUID`
> - `seq_no`
> - `is_valid`
> - and `error_message`
> 
> These columns are not part of the original OpenTelemetry data format and are added by our Parquet collector to simplify certain data operations. To find out more take a look at the [Parquet OTLP collector](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) docs.

***Note:*** The DataFrame below might be too wide for your screen - use its horizontal scroller to see all columns.


---

# Logs signal

Let's begin with the [OpenTelemetry logs signal.](https://opentelemetry.io/docs/specs/otel/logs/)

As the name suggests it contains individual log messages generated by your code and the libraries you're using. Includes timestamps (in the 
`time_unix_nano` and `observed_time_unix_nano` columns), severity level [INFO, WARNING, ERROR...] (in the `severity_number` and `severity_text` columns),
the log message itself (in the `body_*` columns) and additional `attributes` (more about attributes below).

A notable OpenTelemetry extension over traditional logging is the ability to correlate log messages to the traces (`trace_id` column) and spans (`span_id` column) within which they were generated. (`traces` are intoduced later in this notebook.)

Let's jump to exploring some raw `logs.`

> ***Note:*** Raw data displayed below is ***as recorded by*** our [Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet), which adds a few extra columns:
> - `batch_timestamp`
> - `batch_UUID`
> - `seq_no`
> - `is_valid`
> - and `error_message`
> 
> These columns are not part of the original OpenTelemetry data format and are added by our Parquet collector to simplify certain data operations. To find out more take a look at the [Parquet OTLP collector](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) docs.

***Note:*** The DataFrame below might be too wide for your screen - use its horizontal scroller to see all columns.


In [5]:
# Read all Parquet files in the specified directory as a single Pandas DataFrame. Make sure the directory contains only log Parquets.
raw_df = pd.read_parquet('otel-demo-app/logs_raw/', engine='pyarrow')
# Show some data
raw_df.head(25).fillna('')

,batch_timestamp,batch_UUID,seq_no,resource_attributes,resource_dropped_attributes_count,resource_schema_url,scope_name,scope_version,scope_attributes,scope_dropped_attributes_count,time_unix_nano,observed_time_unix_nano,severity_number,severity_text,body_type,body_string,body_bool,body_int,body_double,body_array,body_kvlist,body_bytes,attributes,dropped_attributes_count,flags,trace_id,span_id,log_schema_url,is_valid,error_message
0,1723626938584,7d3116f7-be82-47cb-8c71-1e7b63dfe62b,0,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.sdk._logs._internal,,[],0,1723626933429660416,1723626933429750407,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,"Recommendation service started, listening on port 8080",,,,,,,"[{'key': 'otelSpanID', 'value': {'string_value': '0', 'bool_value': None, 'int_value': None, 'do...",0,0,b'',b'',,True,
1,1723626938772,6284e3b9-642f-42b5-911e-5cd9032d8943,0,"[{'key': 'container.id', 'value': {'string_value': '21ff29407d6923d0abc595bfc6add4e77bb6a423dfbd...",0,https://opentelemetry.io/schemas/1.24.0,org.apache.kafka.common.utils.LoggingSignalHandler,,[],0,1723626938222749152,1723626938222760652,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,"Registered signal handlers for TERM, INT, HUP",,,,,,,[],0,0,b'',b'',,True,
2,1723626938772,6284e3b9-642f-42b5-911e-5cd9032d8943,1,"[{'key': 'container.id', 'value': {'string_value': '21ff29407d6923d0abc595bfc6add4e77bb6a423dfbd...",0,https://opentelemetry.io/schemas/1.24.0,kafka.utils.Log4jControllerRegistration$,,[],0,1723626937885882349,1723626937885898237,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,Registered kafka:type=kafka.Log4jController MBean,,,,,,,[],0,0,b'',b'',,True,
3,1723626938772,6284e3b9-642f-42b5-911e-5cd9032d8943,2,"[{'key': 'container.id', 'value': {'string_value': '21ff29407d6923d0abc595bfc6add4e77bb6a423dfbd...",0,https://opentelemetry.io/schemas/1.24.0,kafka.server.ControllerServer,,[],0,1723626938228994268,1723626938229003786,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,[ControllerServer id=1] Starting controller,,,,,,,[],0,0,b'',b'',,True,
4,1723626938772,6284e3b9-642f-42b5-911e-5cd9032d8943,3,"[{'key': 'container.id', 'value': {'string_value': '21ff29407d6923d0abc595bfc6add4e77bb6a423dfbd...",0,https://opentelemetry.io/schemas/1.24.0,org.apache.zookeeper.common.X509Util,,[],0,1723626938094034974,1723626938094042403,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,Setting -D jdk.tls.rejectClientInitiatedRenegotiation=true to disable client-initiated TLS reneg...,,,,,,,[],0,0,b'',b'',,True,
5,1723626939174,819ad977-36f4-4eff-830e-a5cdad589bf9,0,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.sdk._logs._internal,,[],0,1723626934066117632,0,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,Instrumentation complete,,,,,,,"[{'key': 'code.filepath', 'value': {'string_value': '/usr/src/app/locustfile.py', 'bool_value': ...",0,0,b'',b'',,True,
6,1723626939174,819ad977-36f4-4eff-830e-a5cdad589bf9,1,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.sdk._logs._internal,,[],0,1723626934081092352,0,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,Starting web interface at http://0.0.0.0:8089 (accepting connections from all network interfaces),,,,,,,"[{'key': 'code.filepath', 'value': {'string_value': '/usr/local/lib/python3.12/site-packages/loc...",0,0,b'',b'',,True,
7,1723626939174,819ad977-36f4-4eff-830e-a5cdad589bf9,2,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.sdk._logs._internal,,[],0,1723626934094995456,0,b'SEVERITY_NUMBER_INFO',INFO,STRING_VALUE,Starting Locust 2.18.2,,,,,,,"[{'key': 'code.filepath', 'value': {'string_value': '/usr/local/lib/python3.12/site-packages/loc...",0,0,b'',b'',,True,
8,1723626939174,819ad977-36f4-4eff-830e-a5cdad589bf9,3,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'p

In [6]:
%%markdown

# Resources, scopes and signal attributes

Before continuing with `logs` and other OpenTelemetry signals let's take a break for a moment and introduce a few concepts common to all signal types - [resources](https://opentelemetry.io/docs/concepts/resources/), [scopes](https://opentelemetry.io/docs/concepts/instrumentation-scope/)
and how, in general, OpenTelemetry adds meta-data (as 'attributes') to its signals.

An OpenTelemetry `resource` is essentially one emitter of telemetry, a running instance of your app. For example - one jvm, one nodejs process, etc. A `scope` is a logical unit within a `resource`, a component of your app - a REST library, a database connector or just a module
that you've developed and would like to treat its telemetry separately.

Within the data, the `resource_attributes` column contains meta-data of the `resource` generating a signal. These attributes are recorded by [resource detectors](https://opentelemetry.io/docs/concepts/resources/#resource-detectors) - OpenTelemetry plugins that know how to extract 
additional information from the specific environment where your application runs - a Kubernetes cluster or a public cloud like AWS, Azure, GCP and other. You can also add extra details through configuration or code.

> Use `resources` meta-data to:
> - 'group' signals originating from the same instance of an app
> - ... or all instances running on the same host
> - ... under the same environment
> - ... and so on

Similarly, `scope_attributes` contains scope-related meta-data. The `scope_name` and `scope_version` columns typically contain the name and version of a module. Data about a `scope` is recorded by 'autoinstrumentation plugins' (OpenTelemetry provides telemetry plugins for many
popular libraries - see the [docs for your programming language](https://opentelemetry.io/docs/languages/) for details). You can also add more meta-data to a scope within your code.

> Use the three `scope` columns to:
> - track database requests across all instances of your app
> - measure REST-request times
> - ...etc

Finally, every emitted signal can have its own attributes - recorded by either you (by setting an attribute in your code) or automatically by a provided 'autoinstrumentation plugin'. These attributes can be, for example, the source file name and line number where a log message
was emitted, a stack trace where an error was encountered, the path of an HTTP request and so on.

Attribute columns in OpenTelemetry data are basically lists of key-value pairs and as such are a bit difficult to work with. In these example notebooks we'll use a simple python function to extract all key-value pairs and turn them into individual columns - using the key as a column name:


# Resources, scopes and signal attributes

Before continuing with `logs` and other OpenTelemetry signals let's take a break for a moment and introduce a few concepts common to all signal types - [resources](https://opentelemetry.io/docs/concepts/resources/), [scopes](https://opentelemetry.io/docs/concepts/instrumentation-scope/)
and how, in general, OpenTelemetry adds meta-data (as 'attributes') to its signals.

An OpenTelemetry `resource` is essentially one emitter of telemetry, a running instance of your app. For example - one jvm, one nodejs process, etc. A `scope` is a logical unit within a `resource`, a component of your app - a REST library, a database connector or just a module
that you've developed and would like to treat its telemetry separately.

Within the data, the `resource_attributes` column contains meta-data of the `resource` generating a signal. These attributes are recorded by [resource detectors](https://opentelemetry.io/docs/concepts/resources/#resource-detectors) - OpenTelemetry plugins that know how to extract 
additional information from the specific environment where your application runs - a Kubernetes cluster or a public cloud like AWS, Azure, GCP and other. You can also add extra details through configuration or code.

> Use `resources` meta-data to:
> - 'group' signals originating from the same instance of an app
> - ... or all instances running on the same host
> - ... under the same environment
> - ... and so on

Similarly, `scope_attributes` contains scope-related meta-data. The `scope_name` and `scope_version` columns typically contain the name and version of a module. Data about a `scope` is recorded by 'autoinstrumentation plugins' (OpenTelemetry provides telemetry plugins for many
popular libraries - see the [docs for your programming language](https://opentelemetry.io/docs/languages/) for details). You can also add more meta-data to a scope within your code.

> Use the three `scope` columns to:
> - track database requests across all instances of your app
> - measure REST-request times
> - ...etc

Finally, every emitted signal can have its own attributes - recorded by either you (by setting an attribute in your code) or automatically by a provided 'autoinstrumentation plugin'. These attributes can be, for example, the source file name and line number where a log message
was emitted, a stack trace where an error was encountered, the path of an HTTP request and so on.

Attribute columns in OpenTelemetry data are basically lists of key-value pairs and as such are a bit difficult to work with. In these example notebooks we'll use a simple python function to extract all key-value pairs and turn them into individual columns - using the key as a column name:


In [7]:
# Given an 'attributes' column value - return individual key-values as separate columns
def otel_attrs(series):
    def get_val(v):
        if v['bytes_value'] is not None:
            return v['bytes_value']
        elif v['kvlist_value'] is not None:
            # ignore these for simplicity - dicts are difficult to compare/search for
            return 'ignoring attribute dict value'
        elif v['array_value'] is not None:
            # same as above
            return 'ignoring attribute list value'
        elif v['double_value'] is not None:
            return v['double_value']
        elif v['int_value'] is not None:
            return v['int_value']
        elif v['bool_value'] is not None:
            return v['bool_value']
        elif v['string_value'] is not None:
            return v['string_value']
        else:
            return None

    keys = [x['key'] for x in series.iloc[0]]
    if not keys:
        return pd.Series([], dtype=pd.StringDtype())
    else:
        values = [get_val(x['value']) for x in series.iloc[0]]
        return pd.Series(values, index=keys)

In [8]:
# Now, apply the above function to all 'attributes' columns in the logs DataFrame
resource_attrs_df=raw_df[['resource_attributes']].apply(otel_attrs, axis=1).add_prefix('ra_')
scope_attrs_df=raw_df[['scope_attributes']].apply(otel_attrs, axis=1).add_prefix('sa_')
attrs_df=raw_df[['attributes']].apply(otel_attrs, axis=1).add_prefix('a_')

In [9]:
%%markdown

## Explore logs resource meta-data

Below you can see some examples of `resource` meta-data, as individual columns extracted from the `resource_attributes` column.

You can see details about the Kubernetes environment, operating system, services, process, programming languages, etc.

Note that individual rows may have a different set of populated columns, as different programming languages have different `resource detectors`. 


## Explore logs resource meta-data

Below you can see some examples of `resource` meta-data, as individual columns extracted from the `resource_attributes` column.

You can see details about the Kubernetes environment, operating system, services, process, programming languages, etc.

Note that individual rows may have a different set of populated columns, as different programming languages have different `resource detectors`. 


In [10]:
resource_attrs_df.head(25).fillna('')

,ra_container.id,ra_host.arch,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.name,ra_os.type,ra_os.version,ra_process.command,ra_process.command_args,ra_process.command_line,ra_process.executable.path,ra_process.owner,ra_process.pid,ra_process.runtime.description,ra_process.runtime.name,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.distro.name,ra_telemetry.distro.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
0,,,,opentelemetry-demo-recommendationservice,otel-demo-parquet,minikube,10.244.1.52,opentelemetry-demo-recommendationservice-64768c6df6-2wtjl,2024-08-14T09:15:31Z,b3d5bd61-c923-4f56-86b8-50f347fdbd78,,,,,,,,,,,,,,b3d5bd61-c923-4f56-86b8-50f347fdbd78,recommendationservice,opentelemetry-demo-parquet,1.11.1,,,python,opentelemetry,1.25.0
1,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
2,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
3,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
4,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
5,,,,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,,,,,,,,,,,,,,c8245f82-76f9-409b-848b-1f322ab055

In [11]:
%%markdown

#### Here are the individual programming languages that reported logs:


#### Here are the individual programming languages that reported logs:


In [12]:
resource_attrs_df[['ra_telemetry.sdk.language']].drop_duplicates()

,ra_telemetry.sdk.language
0,python
1,java
146,dotnet
152,cpp
852,php


In [13]:
%%markdown
#### ...a few resource attributes emitted by python:

#### ...a few resource attributes emitted by python:


In [14]:
resource_attrs_df[resource_attrs_df['ra_telemetry.sdk.language'] == 'python'].head(5).dropna(axis=1, how='all')

,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
0,opentelemetry-demo-recommendationservice,otel-demo-parquet,minikube,10.244.1.52,opentelemetry-demo-recommendationservice-64768c6df6-2wtjl,2024-08-14T09:15:31Z,b3d5bd61-c923-4f56-86b8-50f347fdbd78,b3d5bd61-c923-4f56-86b8-50f347fdbd78,recommendationservice,opentelemetry-demo-parquet,1.11.1,python,opentelemetry,1.25.0
5,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,python,opentelemetry,1.23.0
6,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,python,opentelemetry,1.23.0
7,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,python,opentelemetry,1.23.0
8,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,python,opentelemetry,1.23.0


In [15]:
%%markdown

#### ...and some more, emitted by java:


#### ...and some more, emitted by java:


In [16]:
resource_attrs_df[resource_attrs_df['ra_telemetry.sdk.language'] == 'java'].head(5).dropna(axis=1, how='all')

,ra_container.id,ra_host.arch,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.type,ra_process.command_line,ra_process.executable.path,ra_process.pid,ra_process.runtime.description,ra_process.runtime.name,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.distro.name,ra_telemetry.distro.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
1,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,linux,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
2,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,linux,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
3,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,linux,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
4,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,linux,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
11,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,linux,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,o

In [17]:
%%markdown

#### The distinct (by name) scopes in our logs data:


#### The distinct (by name) scopes in our logs data:


In [18]:
raw_df[['scope_name']].drop_duplicates()

,scope_name
0,opentelemetry.sdk._logs._internal
1,org.apache.kafka.common.utils.LoggingSignalHandler
2,kafka.utils.Log4jControllerRegistration$
3,kafka.server.ControllerServer
4,org.apache.zookeeper.common.X509Util
11,kafka.server.NodeToControllerRequestThread
21,kafka.raft.KafkaMetadataLog$
22,org.apache.kafka.common.utils.AppInfoParser
25,kafka.raft.TimingWheelExpirationService$ExpiredOperationReaper
26,org.apache.kafka.raft.QuorumState


In [19]:
%%markdown

#### ...and example logs of one of the scopes (the fraud detection demo microservice):


#### ...and example logs of one of the scopes (the fraud detection demo microservice):


In [20]:
raw_df[raw_df['scope_name'] == 'frauddetectionservice'][['observed_time_unix_nano', 'severity_text', 'body_string']].head(10)

,observed_time_unix_nano,severity_text,body_string
873,1723626979939766881,INFO,"Consumed record with orderId: ddaaaf61-5a1d-11ef-ac87-9e6131d66c8e, and updated total count to: 1"
1029,1723627032386865386,INFO,"Consumed record with orderId: fd57826a-5a1d-11ef-ac87-9e6131d66c8e, and updated total count to: 2"
1045,1723627034196975395,INFO,"Consumed record with orderId: fe68f06e-5a1d-11ef-ac87-9e6131d66c8e, and updated total count to: 3"
1287,1723627071462831737,INFO,"Consumed record with orderId: 0a951e83-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 4"
2535,1723627310413152876,INFO,"Consumed record with orderId: 483e066c-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 5"
2536,1723627310417941710,INFO,"Consumed record with orderId: 3afa16ba-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 6"
2537,1723627310418373499,INFO,"Consumed record with orderId: 72ee62e3-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 7"
2551,1723627311750640939,INFO,"Consumed record with orderId: 92e0c7dd-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 8"
2552,1723627311752001327,INFO,"Consumed record with orderId: 804c1d2d-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 9"
2568,1723627315980693436,INFO,"Consumed record with orderId: 3a08999f-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 10"


In [21]:
%%markdown

#### ...or the Apache Kafka logs:


#### ...or the Apache Kafka logs:


In [22]:
raw_df[resource_attrs_df['ra_service.name'] == 'kafka'][['observed_time_unix_nano', 'severity_text', 'body_string']]

,observed_time_unix_nano,severity_text,body_string
1,1723626938222760652,INFO,"Registered signal handlers for TERM, INT, HUP"
2,1723626937885898237,INFO,Registered kafka:type=kafka.Log4jController MBean
3,1723626938229003786,INFO,[ControllerServer id=1] Starting controller
4,1723626938094042403,INFO,Setting -D jdk.tls.rejectClientInitiatedRenegotiation=true to disable client-initiated TLS reneg...
11,1723626939137715614,INFO,[controller-1-to-controller-registration-channel-manager]: Starting
12,1723626939138571296,INFO,"[controller-1-to-controller-registration-channel-manager]: Recorded new controller, from now on ..."
13,1723626939167687867,INFO,[broker-1-to-controller-forwarding-channel-manager]: Starting
14,1723626939167924714,INFO,"[broker-1-to-controller-forwarding-channel-manager]: Recorded new controller, from now on will u..."
15,1723626939207830541,INFO,[broker-1-to-controller-alter-partition-channel-manager]: Starting
16,1723626939208146097,INFO,"[broker-1-to-controller-alter-partition-channel-manager]: Recorded new controller, from now on w..."


In [23]:
%%markdown

#### Individual log message attributes

As noted above, each `logs` message may contain additional metadata:
    
> ***Hint:*** use the horizontal scroller below


#### Individual log message attributes

As noted above, each `logs` message may contain additional metadata:
    
> ***Hint:*** use the horizontal scroller below


In [24]:
attrs_df.head(25).fillna('')

,a_ContentRoot,a_EnvName,a_address,a_code.filepath,a_code.function,a_code.lineno,a_context,a_exception.message,a_exception.stacktrace,a_exception.type,a_http,a_https,a_otelServiceName,a_otelSpanID,a_otelTraceID,a_otelTraceSampled,a_productId,a_quantity,a_urls,a_userId
0,,,,/usr/src/app/recommendation_server.py,<module>,172.0,,,,,,,recommendationservice,0,0,False,,,,
1,,,,,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,,,,,
3,,,,,,,,,,,,,,,,,,,,
4,,,,,,,,,,,,,,,,,,,,
5,,,,/usr/src/app/locustfile.py,<module>,69.0,,,,,,,,,,,,,,
6,,,,/usr/local/lib/python3.12/site-packages/locust/main.py,main,307.0,,,,,,,,,,,,,,
7,,,,/usr/local/lib/python3.12/site-packages/locust/main.py,main,507.0,,,,,,,,,,,,,,
8,,,,/usr/local/lib/python3.12/site-packages/locust/main.py,start_automatic_run,419.0,,,,,,,,,,,,,,
9,,,,/usr/local/lib/python3.12/site-packages/locust/runners.py,_start,489.0,,,,,,,,,,,,,,


In [25]:
%%markdown

#### Python, for example, adds attributes for the source file name, function and line number:


#### Python, for example, adds attributes for the source file name, function and line number:


In [26]:
attrs_df[attrs_df['a_code.filepath'].notna()].head(25).dropna(axis=1, how='all')

,a_code.filepath,a_code.function,a_code.lineno,a_exception.message,a_exception.stacktrace,a_exception.type,a_otelServiceName,a_otelSpanID,a_otelTraceID,a_otelTraceSampled
0,/usr/src/app/recommendation_server.py,<module>,172.0,NaN,NaN,NaN,recommendationservice,0,0,False
5,/usr/src/app/locustfile.py,<module>,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,/usr/local/lib/python3.12/site-packages/locust/main.py,main,307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,/usr/local/lib/python3.12/site-packages/locust/main.py,main,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,/usr/local/lib/python3.12/site-packages/locust/main.py,start_automatic_run,419.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,/usr/local/lib/python3.12/site-packages/locust/runners.py,_start,489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,/usr/local/lib/python3.12/site-packages/openfeature/client.py,evaluate_flag_details,364.0,received grpc status code StatusCode.UNAVAILABLE,"Traceback (most recent call last):\n File ""/usr/local/lib/python3.12/site-packages/openfeature/...",GeneralError,NaN,NaN,NaN,NaN
157,/usr/src/app/recommendation_server.py,ListRecommendations,47.0,NaN,NaN,NaN,recommendationservice,09f9aa6c3369abcf,d0d72da1bbd44c36eaeddfd50bc08716,True
158,/usr/src/app/recommendation_server.py,ListRecommendations,47.0,NaN,NaN,NaN,recommendationservice,ff5f08f07dc36b87,ef12a39efadfe10b568585a0f85c4185,True
159,/usr/src/app/recommendation_server.py,ListRecommendations,47.0,NaN,NaN,NaN,recommendationservice,508ecb1395bd53ca,3300b293bf9dc71fb8570216a3dd6a55,True


In [27]:
%%markdown

#### Your code can attach custom attributes

The [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) adds details about a product id, quantity and a user id: 


#### Your code can attach custom attributes

The [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) adds details about a product id, quantity and a user id: 


In [28]:
attrs_df[attrs_df['a_productId'].notna()][['a_productId', 'a_quantity', 'a_userId']].head(10)

,a_productId,a_quantity,a_userId
866,66VCHSJNUP,4.0,dd867ea0-5a1d-11ef-b321-32eb2a9fa2f1
868,9SIQT8TOJO,2.0,dd867ea0-5a1d-11ef-b321-32eb2a9fa2f1
1019,OLJCESPC7Z,3.0,fbc2cc0c-5a1d-11ef-88c4-32eb2a9fa2f1
1049,66VCHSJNUP,3.0,fd507b14-5a1d-11ef-88c4-32eb2a9fa2f1
1051,9SIQT8TOJO,2.0,fd507b14-5a1d-11ef-88c4-32eb2a9fa2f1
1057,1YMWWN1N4O,2.0,fe639748-5a1d-11ef-88c4-32eb2a9fa2f1
1100,LS4PSXUNUM,4.0,008f200a-5a1e-11ef-88c4-32eb2a9fa2f1
1181,9SIQT8TOJO,10.0,0a8ff8ae-5a1e-11ef-88c4-32eb2a9fa2f1
1280,1YMWWN1N4O,3.0,13d73166-5a1e-11ef-88c4-32eb2a9fa2f1
1333,2ZYFJ3GM2N,1.0,17d6738a-5a1e-11ef-88c4-32eb2a9fa2f1


In [29]:
%%markdown

# Transforming logs data

Let's reformat the raw `logs` data to something easier to use. Below, we'll extract only a few meaningful columns from the raw data and 'join' 
the `resource`, `scope` and entry attributes to form a new DataFrame.

***Note:*** As mentioned above, the [Parquet Collector](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) adds extra
columns. We'll use one of them - `is_valid` to filter out records that were received in error.


# Transforming logs data

Let's reformat the raw `logs` data to something easier to use. Below, we'll extract only a few meaningful columns from the raw data and 'join' 
the `resource`, `scope` and entry attributes to form a new DataFrame.

***Note:*** As mentioned above, the [Parquet Collector](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) adds extra
columns. We'll use one of them - `is_valid` to filter out records that were received in error.


In [30]:
# Grab only some columns
base_df=raw_df.loc[raw_df['is_valid'] == True][
    [
        'scope_name',
        'scope_version',
        'time_unix_nano',
        'observed_time_unix_nano',
        'severity_number',
        'severity_text',
        'flags',
        'trace_id',
        'span_id'
    ]
]
# Here, we're not interested in log messages that are not strings, so, only get the string-valued messages
base_df['body'] = raw_df.loc[raw_df['body_type'] == 'STRING_VALUE']['body_string']

In [31]:
base_df.head(25)

,scope_name,scope_version,time_unix_nano,observed_time_unix_nano,severity_number,severity_text,flags,trace_id,span_id,body
0,opentelemetry.sdk._logs._internal,,1723626933429660416,1723626933429750407,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',"Recommendation service started, listening on port 8080"
1,org.apache.kafka.common.utils.LoggingSignalHandler,,1723626938222749152,1723626938222760652,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',"Registered signal handlers for TERM, INT, HUP"
2,kafka.utils.Log4jControllerRegistration$,,1723626937885882349,1723626937885898237,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Registered kafka:type=kafka.Log4jController MBean
3,kafka.server.ControllerServer,,1723626938228994268,1723626938229003786,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',[ControllerServer id=1] Starting controller
4,org.apache.zookeeper.common.X509Util,,1723626938094034974,1723626938094042403,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Setting -D jdk.tls.rejectClientInitiatedRenegotiation=true to disable client-initiated TLS reneg...
5,opentelemetry.sdk._logs._internal,,1723626934066117632,0,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Instrumentation complete
6,opentelemetry.sdk._logs._internal,,1723626934081092352,0,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Starting web interface at http://0.0.0.0:8089 (accepting connections from all network interfaces)
7,opentelemetry.sdk._logs._internal,,1723626934094995456,0,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Starting Locust 2.18.2
8,opentelemetry.sdk._logs._internal,,1723626934095175168,0,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',"No run time limit set, use CTRL+C to interrupt"
9,opentelemetry.sdk._logs._internal,,1723626934102390528,0,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Ramping to 10 users at a rate of 1.00 per second


In [32]:
%%markdown

#### Now 'join' the base columns above with all attributes:


#### Now 'join' the base columns above with all attributes:


In [33]:
ext_df=base_df.join(attrs_df).join(resource_attrs_df).join(scope_attrs_df)
ext_df.head(25).fillna('')

,scope_name,scope_version,time_unix_nano,observed_time_unix_nano,severity_number,severity_text,flags,trace_id,span_id,body,a_ContentRoot,a_EnvName,a_address,a_code.filepath,a_code.function,a_code.lineno,a_context,a_exception.message,a_exception.stacktrace,a_exception.type,a_http,a_https,a_otelServiceName,a_otelSpanID,a_otelTraceID,a_otelTraceSampled,a_productId,a_quantity,a_urls,a_userId,ra_container.id,ra_host.arch,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.name,ra_os.type,ra_os.version,ra_process.command,ra_process.command_args,ra_process.command_line,ra_process.executable.path,ra_process.owner,ra_process.pid,ra_process.runtime.description,ra_process.runtime.name,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.distro.name,ra_telemetry.distro.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
0,opentelemetry.sdk._logs._internal,,1723626933429660416,1723626933429750407,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',"Recommendation service started, listening on port 8080",,,,/usr/src/app/recommendation_server.py,<module>,172.0,,,,,,,recommendationservice,0,0,False,,,,,,,,opentelemetry-demo-recommendationservice,otel-demo-parquet,minikube,10.244.1.52,opentelemetry-demo-recommendationservice-64768c6df6-2wtjl,2024-08-14T09:15:31Z,b3d5bd61-c923-4f56-86b8-50f347fdbd78,,,,,,,,,,,,,,b3d5bd61-c923-4f56-86b8-50f347fdbd78,recommendationservice,opentelemetry-demo-parquet,1.11.1,,,python,opentelemetry,1.25.0
1,org.apache.kafka.common.utils.LoggingSignalHandler,,1723626938222749152,1723626938222760652,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',"Registered signal handlers for TERM, INT, HUP",,,,,,,,,,,,,,,,,,,,,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
2,kafka.utils.Log4jControllerRegistration$,,1723626937885882349,1723626937885898237,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',Registered kafka:type=kafka.Log4jController MBean,,,,,,,,,,,,,,,,,,,,,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kafka.jsa -Xlog:gc...,/opt/java/openjdk/bin/java,,1.0,Eclipse Adoptium OpenJDK 64-Bit Server VM 21.0.2+13-LTS,OpenJDK Runtime Environment,21.0.2+13-LTS,8f20a94b-32ab-4edb-9572-83d9a2f33d4f,kafka,opentelemetry-demo-parquet,1.11.1,opentelemetry-java-instrumentation,2.4.0,java,opentelemetry,1.38.0
3,kafka.server.ControllerServer,,1723626938228994268,1723626938229003786,b'SEVERITY_NUMBER_INFO',INFO,0,b'',b'',[ControllerServer id=1] Starting controller,,,,,,,,,,,,,,,,,,,,,21ff29407d6923d0abc595bfc6add4e77bb6a423dfbda6c4cb6902d40ca9825c,amd64,opentelemetry-demo-kafka-79c66d6644-sm2k4,opentelemetry-demo-kafka,otel-demo-parquet,minikube,10.244.1.47,opentelemetry-demo-kafka-79c66d6644-sm2k4,2024-08-14T09:15:29Z,4076eb9d-ad8c-419c-8905-76a78572ede1,Linux 5.14.0-427.24.1.el9_4.x86_64,,linux,,,,/opt/java/openjdk/bin/java -Xmx400M -Xms400M -XX:SharedArchiveFile=/opt/kafka/kaf

In [34]:
%%markdown

# Correlating log messages

Earlier, we noted one great improvement over traditional logging: OpenTelemetry's ability to correlate signals emitted within the same `trace`,
even as the `trace` 'propagates' across separate microservices, running on different cluster nodes.

The `trace` signal will be introduced in more detail later, but for now, consider the `trace_id` as way of grouping together `logs` signals emitted by 
separate instances working together on the same request.

In the exapmle below, it's a user's shopping cart request, that required various operations like currency convertion, a price quote and fraud analysis.
As the [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) microservices were calling each other to fulfill the business logic behind the user's request - each one of them 'attached' the same `trace_id`
to its `logs`. This mechanism is known as [Context Propagation.](https://opentelemetry.io/docs/concepts/context-propagation/)

Thanks to it, we can now see all log messages related to the same user's request in one single view, despite that they were emitted from completely different 
environments:


# Correlating log messages

Earlier, we noted one great improvement over traditional logging: OpenTelemetry's ability to correlate signals emitted within the same `trace`,
even as the `trace` 'propagates' across separate microservices, running on different cluster nodes.

The `trace` signal will be introduced in more detail later, but for now, consider the `trace_id` as way of grouping together `logs` signals emitted by 
separate instances working together on the same request.

In the exapmle below, it's a user's shopping cart request, that required various operations like currency convertion, a price quote and fraud analysis.
As the [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) microservices were calling each other to fulfill the business logic behind the user's request - each one of them 'attached' the same `trace_id`
to its `logs`. This mechanism is known as [Context Propagation.](https://opentelemetry.io/docs/concepts/context-propagation/)

Thanks to it, we can now see all log messages related to the same user's request in one single view, despite that they were emitted from completely different 
environments:


In [35]:
# lets keep this data frame for a later example
trace_id = b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91'
trace_logs = ext_df[ext_df['trace_id'] == trace_id][[
    'ra_k8s.pod.name',
    'ra_k8s.pod.ip',
    'ra_service.name',
    'ra_telemetry.sdk.language',
    'scope_name',
    'observed_time_unix_nano',
    'severity_text',
    'body',
    'a_userId',
    'trace_id',
    'span_id']]

trace_logs.fillna('')

,ra_k8s.pod.name,ra_k8s.pod.ip,ra_service.name,ra_telemetry.sdk.language,scope_name,observed_time_unix_nano,severity_text,body,a_userId,trace_id,span_id
1605,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,10.244.1.34,currencyservice,cpp,currencyservice,1723627134102665414,INFO,Convert conversion successful,,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\xac\x05@P\x17\xa4\xab\xe7'
1606,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,10.244.1.34,currencyservice,cpp,currencyservice,1723627134104169898,INFO,Convert conversion successful,,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\x83d\xf4\xbe\x84\xb6\xf9A'
1607,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,10.244.1.34,currencyservice,cpp,currencyservice,1723627134106806444,INFO,Convert conversion successful,,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\x8c\xb4\xda\xcb\x12\x85\xa6f'
1608,opentelemetry-demo-quoteservice-787985cdb9-hdlfz,10.244.1.51,quoteservice,php,slim-app,1723627134106146048,INFO,Calculated quote,,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\xaa\xb6\xba-\x11-{\xe0'
1633,opentelemetry-demo-cartservice-6787d8bc8c-qwnj7,10.244.1.40,cartservice,dotnet,cartservice.cartstore.ValkeyCartStore,1723627134101558000,Information,GetCartAsync called with userId={userId},39fe30ba-5a1e-11ef-88c4-32eb2a9fa2f1,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\x04\xbe\xa8\xb3\xaa\xf4<$'
2568,opentelemetry-demo-frauddetectionservice-6bc9d4d9f9-6sssc,10.244.1.43,frauddetectionservice,java,frauddetectionservice,1723627315980693436,INFO,"Consumed record with orderId: 3a08999f-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 10",,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\xf7\xf1\xd2\xf1\xba\xeb4\xfc'
2572,opentelemetry-demo-cartservice-6787d8bc8c-qwnj7,10.244.1.40,cartservice,dotnet,cartservice.cartstore.ValkeyCartStore,1723627315972342200,Information,EmptyCartAsync called with userId={userId},39fe30ba-5a1e-11ef-88c4-32eb2a9fa2f1,b'cV\x84$\xcd\xb1\x05\x82v\xdcl\xab\xbf\xc8\x0c\x91',b'\xb4A-n1\x1b\\\xb1'


In [36]:
%%markdown

### Finally, let's save the pre-processed logs for later use


### Finally, let's save the pre-processed logs for later use


In [37]:
# convert the column data types to something more 'native'
ext_df=ext_df.convert_dtypes()

In [38]:
%%markdown

##### Deduplicate

***Note:*** Logs may be retransmitted (due to network errors, for example). Retransmissions may cause the same `log` entry to be recorded twice. For futher processing it's good to deduplicate the data to get rid of potential retransmits.


##### Deduplicate

***Note:*** Logs may be retransmitted (due to network errors, for example). Retransmissions may cause the same `log` entry to be recorded twice. For futher processing it's good to deduplicate the data to get rid of potential retransmits.


In [39]:
# deduplicate
ext_df=ext_df.drop_duplicates()

In [40]:
# and finally, save to a preprocessed Parquet file
ext_df.to_parquet('otel-demo-app/logs.parquet', engine='pyarrow')

In [41]:
%%markdown

---

# Metrics signal

Moving away from `logs` now and into the next OpenTelemetry signal type - [metrics.](https://opentelemetry.io/docs/specs/otel/metrics/)

Unlike `logs`, which are single events recorded at a fixed point in time, `metrics` are observed over a configurable time interval and reported only once 
for that interval. Typically, the reported value will be 'aggregated' in some way - like to a sum, or a histogram (as seen in the `type` column).

The time interval will be reported in columns `start_time_unix_nano` and `time_unix_nano`. However, their exact interpretation may change depending on the 
`aggregation_temporality` and possibly `is_monotonic` columns.

How to properly extract values out of `metrics` datasets is covered in a separate notebook. Before heading to it though, take a look at the basics examples below.
For a quick glimpse of what's typically measured (and how) go through the `name`, `description`, `unit` and `type` columns. Depending on `type` you can find the 
repoted value in the `gauge_*`, `sum_*`, `histogram_*`, `exponential_histogram_*` and `summary_*` columns. In `metrics` data you will also find `resource` and 
`scope` meta-data, as well as `attributes` for each individual data point (for more on these look at the `logs` examples above).

The [OTLP Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) does a bit of unpacking of the OpenTelemetry 
data that it receives (basically extracting each data point of each `metric`, of each `scope` and each `resource`). As it does so, a number of columns are added:
- `batch_timestamp`
- `batch_UUID`
- `seq_no`
- `datapoint_seq_no`
- `is_valid`
- `error_message`

These columns are not part of the original OpenTelemetry schema, they're only added to help with some more complex operations on the data.

We'll now drop the `logs` DataFrame used above and replace it with the raw `metrics.` Then, in a similar way we'll extract all attributes:


---

# Metrics signal

Moving away from `logs` now and into the next OpenTelemetry signal type - [metrics.](https://opentelemetry.io/docs/specs/otel/metrics/)

Unlike `logs`, which are single events recorded at a fixed point in time, `metrics` are observed over a configurable time interval and reported only once 
for that interval. Typically, the reported value will be 'aggregated' in some way - like to a sum, or a histogram (as seen in the `type` column).

The time interval will be reported in columns `start_time_unix_nano` and `time_unix_nano`. However, their exact interpretation may change depending on the 
`aggregation_temporality` and possibly `is_monotonic` columns.

How to properly extract values out of `metrics` datasets is covered in a separate notebook. Before heading to it though, take a look at the basics examples below.
For a quick glimpse of what's typically measured (and how) go through the `name`, `description`, `unit` and `type` columns. Depending on `type` you can find the 
repoted value in the `gauge_*`, `sum_*`, `histogram_*`, `exponential_histogram_*` and `summary_*` columns. In `metrics` data you will also find `resource` and 
`scope` meta-data, as well as `attributes` for each individual data point (for more on these look at the `logs` examples above).

The [OTLP Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) does a bit of unpacking of the OpenTelemetry 
data that it receives (basically extracting each data point of each `metric`, of each `scope` and each `resource`). As it does so, a number of columns are added:
- `batch_timestamp`
- `batch_UUID`
- `seq_no`
- `datapoint_seq_no`
- `is_valid`
- `error_message`

These columns are not part of the original OpenTelemetry schema, they're only added to help with some more complex operations on the data.

We'll now drop the `logs` DataFrame used above and replace it with the raw `metrics.` Then, in a similar way we'll extract all attributes:


In [42]:
# Read all files in a directory (make sure they're only metrics parquets):
raw_df = pd.read_parquet('otel-demo-app/metrics_raw/', engine='pyarrow')
raw_df.head(25).fillna('')

,batch_timestamp,batch_UUID,seq_no,resource_attributes,resource_dropped_attributes_count,resource_schema_url,scope_name,scope_version,scope_attributes,scope_dropped_attributes_count,name,description,unit,type,datapoint_seq_no,attributes,start_time_unix_nano,time_unix_nano,exemplars,flags,gauge_type,gauge_double,gauge_int,sum_type,sum_double,sum_int,histogram_count,histogram_sum,histogram_bucket_counts,histogram_explicit_bounds,histogram_min,histogram_max,exponential_histogram_count,exponential_histogram_sum,exponential_histogram_scale,exponential_histogram_zero_count,exponential_histogram_positive,exponential_histogram_negative,exponential_histogram_min,exponential_histogram_max,exponential_histogram_zero_threshold,summary_count,summary_sum,summary_quantile_values,aggregation_temporality,is_monotonic,metric_schema_url,metric_metadata,is_valid,error_message
0,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,0,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.duration,Measures the duration of the HTTP check.,ms,GAUGE,0,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,AS_INT,0.0,2.0,,,,,,[],[],,,,,,,,,,,,,,[],,,,[],True,
1,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,1,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.error,Records errors occurring during HTTP check.,{error},SUM,0,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,,,,AS_INT,0.0,1.0,,,[],[],,,,,,,,,,,,,,[],b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,[],True,
2,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,2,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,0,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,,,,AS_INT,0.0,0.0,,,[],[],,,,,,,,,,,,,,[],b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,[],True,
3,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,2,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,1,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,,,,AS_INT,0.0,0.0,,,[],[],,,,,,,,,,,,,,[],b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,[],True,
4,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,2,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,2,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,,,,AS_INT,0.0,0.0,,,[],[],,,,,,,,,,,,,,[],b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,[],True,
5,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,2,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,3,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,,,,AS_INT,0.0,0.0,,,[],[],,,,,,,,,,,,,,[],b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,[],True,
6,1723626938249,daaf2f22-874a-4ec9-b3fd-2d1bdd94ef97,2,[],0,,otelcol/httpcheckreceiver,0.105.0,[],0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,4,"[{'key': 'http.url', 'value': {'string_value': 'http://opentelemetry-demo-frontendproxy:8080', '...",1723626930335324612,1723626931381380943,[],0,,,,AS_INT,0.0,0.0,,,[],[],,,,,,,,,,,,,,[],b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,[],True,
7,1723626941995,f8941e51-73fd-4108-b32b-e52541b96b45,0,"[{'key': 'host.name', 'value': {'string_v

In [43]:
# Apply the same otel_attrs function defined at the beginning of this notebook, get the attributes
resource_attrs_df=raw_df[['resource_attributes']].apply(otel_attrs, axis=1).add_prefix('ra_')
scope_attrs_df=raw_df[['scope_attributes']].apply(otel_attrs, axis=1).add_prefix('sa_')
attrs_df=raw_df[['attributes']].apply(otel_attrs, axis=1).add_prefix('a_')

In [44]:
%%markdown

#### Metrics cover aspects like load, usage, amount of work done, number of errors encountered and more:


#### Metrics cover aspects like load, usage, amount of work done, number of errors encountered and more:


In [45]:
raw_df[['name', 'description']].drop_duplicates()

,name,description
0,httpcheck.duration,Measures the duration of the HTTP check.
1,httpcheck.error,Records errors occurring during HTTP check.
2,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0."
7,http.server.duration,Measures the duration of inbound HTTP requests.
10,http.server.response.size,Measures the size of HTTP request messages (compressed).
13,http.server.active_requests,Measures the number of concurrent HTTP requests that are currently in-flight.
17,feature_flag.flagd.impression,Measures the number of evaluations for a given flag.
28,feature_flag.flagd.evaluation.reason,Measures the number of evaluations for a given reason.
39,http.client.duration,measures the duration of the outbound HTTP request
43,system.cpu.time,System CPU time


In [46]:
%%markdown

#### Resource attributes are quite similar to the logs signal above:


#### Resource attributes are quite similar to the logs signal above:


In [47]:
resource_attrs_df.head(25).dropna(axis=1, how='all').dropna(axis=0, how='all').fillna('')

,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.type,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
7,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
8,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
9,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
10,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
11,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
12,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
13,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
14,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
15,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-parquet,minikube,10.244.1.42,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,2024-08-14T09:15:28Z,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,Debian GNU/Linux Debian GNU/Linux 12 (bookworm) (Linux opentelemetry-demo-flagd-6d57f47cb8-t8jgb...,linux,go1.22.5,d5391388-6a2b-4911-b4e4-7ecf5ab5e1bf,flagd,v0.11.1,go,opentelemetry,1.28.0
16,opentelemetry-demo-flagd-6d57f47cb8-t8jgb,opentelemetry-demo-flagd,otel-demo-pa

In [48]:
%%markdown

#### ...but we see a few more systems reporting their metrics:


#### ...but we see a few more systems reporting their metrics:


In [49]:
resource_attrs_df[['ra_telemetry.sdk.language']].drop_duplicates()

,ra_telemetry.sdk.language
0,<NA>
7,go
39,python
1332,cpp
1374,java
1392,ruby
1424,webjs
1434,php
1440,rust
1462,nodejs


In [50]:
%%markdown

#### Also a slightly different list of scopes:


#### Also a slightly different list of scopes:


In [51]:
raw_df[['scope_name']].drop_duplicates()

,scope_name
0,otelcol/httpcheckreceiver
7,flagd
39,opentelemetry.instrumentation.requests
43,opentelemetry.instrumentation.system_metrics
1332,app.currency
1374,spanmetricsconnector
1604,io.opentelemetry.exporters.otlp-http
1610,adservice
1612,io.opentelemetry.sdk.logs
1614,io.opentelemetry.grpc-1.6


In [52]:
%%markdown

#### ...and a much richer set of metric attributes

Including, for example, http protocol attributes, cpu and resource usage, error details and so on:


#### ...and a much richer set of metric attributes

Including, for example, http protocol attributes, cpu and resource usage, error details and so on:


In [53]:
attrs_df.head(25).fillna('')

,a_app.ads.ad_request_type,a_app.ads.ad_response_type,a_app.payment.currency,a_aspnetcore.routing.is_fallback,a_aspnetcore.routing.match_status,a_client-id,a_count,a_cpu,a_currency_code,a_device,a_direction,a_dropped,a_error.message,a_family,a_feature_flag.key,a_feature_flag.provider_name,a_feature_flag.reason,a_feature_flag.variant,a_generation,a_http.flavor,a_http.host,a_http.method,a_http.request.method,a_http.response.status_code,a_http.route,a_http.scheme,a_http.status_class,a_http.status_code,a_http.url,a_jvm.gc.action,a_jvm.gc.name,a_jvm.memory.pool.name,a_jvm.memory.type,a_jvm.thread.daemon,a_jvm.thread.state,a_method,a_net.host.name,a_net.host.port,a_net.peer.name,a_net.peer.port,a_network.protocol.name,a_network.protocol.version,a_network.transport,a_network.type,a_node-id,a_number_of_items,a_operation,a_partition,a_process.cpu.state,a_processorType,a_protocol,a_recommendation.type,a_rpc.grpc.status_code,a_rpc.method,a_rpc.service,a_rpc.system,a_server.address,a_server.port,a_service.name,a_span.kind,a_span.name,a_state,a_status,a_status.code,a_success,a_target,a_topic,a_type,a_url.scheme
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,,,,"Get ""http://opentelemetry-demo-frontendproxy:8080"": dial tcp: lookup opentelemetry-demo-frontend...",,,,,,,,,,,,,,,,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,,,,,,,GET,,,,,1xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,,,,,,,,,,,,,,,,,,,,,GET,,,,,2xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,,,,,,,,,,,,,,,,,,,,,GET,,,,,3xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,,,,,,,,,,,,,,,,,,,,,GET,,,,,4xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,,,,,,,,,,,,,,,,,,,,,,GET,,,,,5xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,,,,,,,,,,,,,,,,,,,,,,POST,,,,,,200,/flagd.evaluation.v1.Service/ResolveAll,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,flagd,,,,,,,,,,
8,,,,,,,,,,,,,,,,,,,,,,POST,,,,,,200,/schema.v1.Service/ResolveBoolean,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,flagd,,,,,,,,,,
9,,,,,,,,,,,,,,,,,,,,,,POST,,,,,,200,/flagd.evaluation.v1.Service/ResolveBoolean,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,flagd,,,,,,,,,,


In [54]:
%%markdown

# Pre-processing metrics data

As with the `logs` above, we'll extract some valuable columns from the raw `metrics` data, join it with extracted attributes and save a new file.

We'll use this new file in another [notebook dedicated to metrics.](metrics.ipynb) 


# Pre-processing metrics data

As with the `logs` above, we'll extract some valuable columns from the raw `metrics` data, join it with extracted attributes and save a new file.

We'll use this new file in another [notebook dedicated to metrics.](metrics.ipynb) 


In [55]:
base_df=raw_df.loc[raw_df['is_valid'] == True][
    [
        'scope_name',
        'scope_version',
        'name',
        'description',
        'unit',
        'type',
        'gauge_type',
        'gauge_int',
        'gauge_double',
        'sum_type',
        'sum_int',
        'sum_double',
        'histogram_count',
        'histogram_sum',
        'histogram_min',
        'histogram_max',
        'histogram_bucket_counts',
        'histogram_explicit_bounds',
        'start_time_unix_nano',
        'time_unix_nano',
        'aggregation_temporality',
        'is_monotonic'
    ]
]

In [56]:
# Join attributes to the base DataFrame
ext_df=base_df.join(attrs_df).join(resource_attrs_df).join(scope_attrs_df)
ext_df.head(25).fillna('')

,scope_name,scope_version,name,description,unit,type,gauge_type,gauge_int,gauge_double,sum_type,sum_int,sum_double,histogram_count,histogram_sum,histogram_min,histogram_max,histogram_bucket_counts,histogram_explicit_bounds,start_time_unix_nano,time_unix_nano,aggregation_temporality,is_monotonic,a_app.ads.ad_request_type,a_app.ads.ad_response_type,a_app.payment.currency,a_aspnetcore.routing.is_fallback,a_aspnetcore.routing.match_status,a_client-id,a_count,a_cpu,a_currency_code,a_device,a_direction,a_dropped,a_error.message,a_family,a_feature_flag.key,a_feature_flag.provider_name,a_feature_flag.reason,a_feature_flag.variant,a_generation,a_http.flavor,a_http.host,a_http.method,a_http.request.method,a_http.response.status_code,a_http.route,a_http.scheme,a_http.status_class,a_http.status_code,a_http.url,a_jvm.gc.action,a_jvm.gc.name,a_jvm.memory.pool.name,a_jvm.memory.type,a_jvm.thread.daemon,a_jvm.thread.state,a_method,a_net.host.name,a_net.host.port,a_net.peer.name,a_net.peer.port,a_network.protocol.name,a_network.protocol.version,a_network.transport,a_network.type,a_node-id,a_number_of_items,a_operation,a_partition,a_process.cpu.state,a_processorType,a_protocol,a_recommendation.type,a_rpc.grpc.status_code,a_rpc.method,a_rpc.service,a_rpc.system,a_server.address,a_server.port,a_service.name,a_span.kind,a_span.name,a_state,a_status,a_status.code,a_success,a_target,a_topic,a_type,a_url.scheme,ra_container.id,ra_host.arch,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.name,ra_os.type,ra_os.version,ra_process.command,ra_process.command_args,ra_process.command_line,ra_process.executable.name,ra_process.executable.path,ra_process.owner,ra_process.pid,ra_process.runtime.description,ra_process.runtime.name,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.auto.version,ra_telemetry.distro.name,ra_telemetry.distro.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
0,otelcol/httpcheckreceiver,0.105.0,httpcheck.duration,Measures the duration of the HTTP check.,ms,GAUGE,AS_INT,2.0,0.0,,,,,,,,[],[],1723626930335324612,1723626931381380943,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,otelcol/httpcheckreceiver,0.105.0,httpcheck.error,Records errors occurring during HTTP check.,{error},SUM,,,,AS_INT,1.0,0.0,,,,,[],[],1723626930335324612,1723626931381380943,b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,,,,,,,,,,,,"Get ""http://opentelemetry-demo-frontendproxy:8080"": dial tcp: lookup opentelemetry-demo-frontend...",,,,,,,,,,,,,,,,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,otelcol/httpcheckreceiver,0.105.0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,,,,AS_INT,0.0,0.0,,,,,[],[],1723626930335324612,1723626931381380943,b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,,,,,,,,,,,,,,,,,,,,,GET,,,,,1xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,otelcol/httpcheckreceiver,0.105.0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,,,,AS_INT,0.0,0.0,,,,,[],[],1723626930335324612,1723626931381380943,b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,,,,,,,,,,,,,,,,,,,,,GET,,,,,2xx,0.0,http://opentelemetry-demo-frontendproxy:8080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,otelcol/httpcheckreceiver,0.105.0,httpcheck.status,"1 if the check resulted in status_code matching the status_class, otherwise 0.",1,SUM,,,,AS_INT,0.0,0.0,,,,,[],[],1723626930335324612,1723626931381380943,b'AGGREGATION_TEMPORALITY_CUMULATIVE',False,,,,,,,,,,,,,,,,,,,,,,GET,,,,,3xx,0.0,h

In [57]:
%%markdown

# Out-of-the-box metrics

Thanks to its large ecosystem of plugins (and auto-instrumentation labraries) OpenTelemetry is capable of providing `metrics` (and in fact other signals) out-of-the-box.

Here's an example of how it collected Apache Kafka `metrics` automatically, with the help of a Java JMX plugin:


# Out-of-the-box metrics

Thanks to its large ecosystem of plugins (and auto-instrumentation labraries) OpenTelemetry is capable of providing `metrics` (and in fact other signals) out-of-the-box.

Here's an example of how it collected Apache Kafka `metrics` automatically, with the help of a Java JMX plugin:


In [58]:
ext_df[ext_df['scope_name'] == 'io.opentelemetry.jmx'][
    [
        'ra_service.name',
        'name',
        'description',
        'unit',
        'type',
        'gauge_int',
        'gauge_double',
        'sum_int',
        'sum_double',
        'start_time_unix_nano',
        'time_unix_nano',
        'aggregation_temporality',
        'a_type'
    ]
].head(25).fillna('')

,ra_service.name,name,description,unit,type,gauge_int,gauge_double,sum_int,sum_double,start_time_unix_nano,time_unix_nano,aggregation_temporality,a_type
3782,kafka,kafka.logs.flush.time.99p,Log flush time - 99th percentile,ms,GAUGE,0.0,37.183494,,,1723626936540681556,1723627056546499014,,
3783,kafka,kafka.logs.flush.time.50p,Log flush time - 50th percentile,ms,GAUGE,0.0,2.668792,,,1723626936540681556,1723627056546499014,,
3784,kafka,kafka.request.queue,Size of the request queue,{requests},SUM,,,0.0,0.0,1723626936540681556,1723627056546499014,b'AGGREGATION_TEMPORALITY_CUMULATIVE',
3785,kafka,kafka.request.time.99p,The 99th percentile time the broker has taken to service requests,ms,GAUGE,0.0,0.0,,,1723626936540681556,1723627056546499014,,FetchFollower
3786,kafka,kafka.request.time.99p,The 99th percentile time the broker has taken to service requests,ms,GAUGE,0.0,30.0,,,1723626936540681556,1723627056546499014,,Produce
3787,kafka,kafka.request.time.99p,The 99th percentile time the broker has taken to service requests,ms,GAUGE,0.0,514.59,,,1723626936540681556,1723627056546499014,,FetchConsumer
3788,kafka,kafka.request.time.total,The total time the broker has taken to service requests,ms,SUM,,,0.0,0.0,1723626936540681556,1723627056546499014,b'AGGREGATION_TEMPORALITY_CUMULATIVE',FetchFollower
3789,kafka,kafka.request.time.total,The total time the broker has taken to service requests,ms,SUM,,,3.0,0.0,1723626936540681556,1723627056546499014,b'AGGREGATION_TEMPORALITY_CUMULATIVE',Produce
3790,kafka,kafka.request.time.total,The total time the broker has taken to service requests,ms,SUM,,,446.0,0.0,1723626936540681556,1723627056546499014,b'AGGREGATION_TEMPORALITY_CUMULATIVE',FetchConsumer
3791,kafka,kafka.network.io,The bytes received or sent by the broker,By,SUM,,,5224.0,0.0,1723626936540681556,1723627056546499014,b'AGGREGATION_TEMPORALITY_CUMULATIVE',


In [59]:
%%markdown

# Custom metrics

Through OpenTelemetry instrumentation APIs your code can emit `metrics` specific to your app.

The [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet)'s currency convertion microservice measures how much of a given currency it has converted:


# Custom metrics

Through OpenTelemetry instrumentation APIs your code can emit `metrics` specific to your app.

The [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet)'s currency convertion microservice measures how much of a given currency it has converted:


In [60]:
ext_df[ext_df['scope_name'] == 'app.currency'][
    [
        'ra_k8s.pod.name',
        'scope_name',
        'name',
        'description',
        'unit',
        'type',
        'sum_type',
        'sum_int',
        'a_currency_code',
        'time_unix_nano',
        'start_time_unix_nano',
        'aggregation_temporality'
    ]
].head(25)

,ra_k8s.pod.name,scope_name,name,description,unit,type,sum_type,sum_int,a_currency_code,time_unix_nano,start_time_unix_nano,aggregation_temporality
1332,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,3.0,USD,1723626989770557713,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
1333,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,56.0,CHF,1723626989770557713,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
3259,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,104.0,CHF,1723627049771120249,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
3260,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,8.0,USD,1723627049771120249,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
4199,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,10.0,USD,1723627109772061981,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
4200,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,164.0,CHF,1723627109772061981,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
5937,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,47.0,USD,1723627709778022058,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
5938,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,374.0,CHF,1723627709778022058,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
5939,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,9.0,CAD,1723627709778022058,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'
6266,opentelemetry-demo-currencyservice-597bbd47c4-dj9g2,app.currency,app.currency_counter,,,SUM,AS_INT,9.0,CAD,1723627769778783782,1723626929769926474,b'AGGREGATION_TEMPORALITY_CUMULATIVE'


In [61]:
%%markdown

# Correlating metrics

As mentioned earlier, a great feature of OpenTelemetry is the ability to ***correlate signals,*** allowing you to dig deeper into the behaviour of your apps.

We introduced `logs` correlation earlier and we'll show now how `metrics` can be correlated too.

`Metrics` correlation is a bit harder though. `Logs`, being one-off events, are easily mapped 1:1 to the current `trace`. `Metrics`, on the other hand, are updated 
multiple times, possibly within multiple `traces`, or even outside of any `trace`. So, they have to be mapped in a more N:M fashion.
For this reason, OpenTelemetry can add [Exemplars](https://opentelemetry.io/docs/specs/otel/metrics/data-model/#exemplars) to its `metrics` reporting.
These exemplars are basically individual exact values selected during the reporting interval and their main purpose is to record 
`trace_id`s and `span_id`s that can later be used for signal correaltion.

Exemplars are written in the `exemplars` column and contain an array of objects.

Let's take the `rpc.server.duration` metric, as reported by **gRPC**, and 'explode' the list of `exemplars` into extra rows for a quick view:


# Correlating metrics

As mentioned earlier, a great feature of OpenTelemetry is the ability to ***correlate signals,*** allowing you to dig deeper into the behaviour of your apps.

We introduced `logs` correlation earlier and we'll show now how `metrics` can be correlated too.

`Metrics` correlation is a bit harder though. `Logs`, being one-off events, are easily mapped 1:1 to the current `trace`. `Metrics`, on the other hand, are updated 
multiple times, possibly within multiple `traces`, or even outside of any `trace`. So, they have to be mapped in a more N:M fashion.
For this reason, OpenTelemetry can add [Exemplars](https://opentelemetry.io/docs/specs/otel/metrics/data-model/#exemplars) to its `metrics` reporting.
These exemplars are basically individual exact values selected during the reporting interval and their main purpose is to record 
`trace_id`s and `span_id`s that can later be used for signal correaltion.

Exemplars are written in the `exemplars` column and contain an array of objects.

Let's take the `rpc.server.duration` metric, as reported by **gRPC**, and 'explode' the list of `exemplars` into extra rows for a quick view:


In [62]:
exemplars_df=raw_df[
    (raw_df['name'] == 'rpc.server.duration') & (raw_df['exemplars'].apply(lambda e: e.size) > 0)
].sort_values(by=['start_time_unix_nano', 'time_unix_nano'])[
    [
        'scope_name', 'name', 'description', 'unit', 'type', 'start_time_unix_nano', 'time_unix_nano', 'exemplars'       
    ]
].explode('exemplars')

exemplars_df.head(10)

,scope_name,name,description,unit,type,start_time_unix_nano,time_unix_nano,exemplars
1616,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723626990691310448,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
3506,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627050690892769,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
3506,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627050690892769,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
4449,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627110690872218,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
5258,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627110690872218,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
33851,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627170691991446,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
34807,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627230690873998,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
35764,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627290690861235,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
35764,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627290690861235,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."
47509,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627290690861235,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'..."


In [63]:
%%markdown

#### So, what's in an exemplar?

An exemplar records an exact value (in `as_double` or `as_int`, depending on the metric unit type), at a specific time (`time_unix_nano`), during a specific `trace_id` and `span_id`:


#### So, what's in an exemplar?

An exemplar records an exact value (in `as_double` or `as_int`, depending on the metric unit type), at a specific time (`time_unix_nano`), during a specific `trace_id` and `span_id`:


In [64]:
pd.DataFrame(exemplars_df['exemplars'].apply(pd.Series)).head(10)

,filtered_attributes,time_unix_nano,as_double,as_int,span_id,trace_id
1616,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723626941986000000,240.269815,None,"b'\xf0\x9fc\xf6\xda""\x1c\x15'",b'\xa64[\x94<\xeb\x88\xb70!\xb3\xb2z\xd2U\xa2'
3506,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627036217000000,3.029536,None,b'\xd0[&7\x05O$\xc4',"b'c\xab\xb1\xa3""\xec\xa1koN\x93C\x91\x05\xad\x89'"
3506,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627016538000000,28.312800,None,b'\xe7\x92I\x99\x97gr\xc1',b'\x94{\xa8\xe3?\xea\xc2~\xeeK\xf8\x96\x91+J\xd0'
4449,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627096908000000,1.966614,None,b'\x07\xa5\x1b\xc8\xa9l\x88m',b'Z\x04\xd7\xc23d3b\x96\xc8eC7\xcc\x8d\x85'
5258,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627096908000000,1.966614,None,b'\x07\xa5\x1b\xc8\xa9l\x88m',b'Z\x04\xd7\xc23d3b\x96\xc8eC7\xcc\x8d\x85'
33851,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627133368000000,2.024193,None,b'E\xb6&\xf1\xc7\xe6Z0',b'\xadx\x15?\x96\x1b\xae\x0c2f\xab\x879.\x14\xdc'
34807,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627174932000000,2.099705,None,b'\x81s\xbeYI\xef5\x01',b'x\xdd\xe8\xe6\xf3\x8f\xe7\xbc\xcf`\xd5\xbea\x00\x10\x8e'
35764,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627278080000000,4.773562,None,b'P\x97jq\x13\xc1\xe1\xfe',b'8\xb1\xfd\xf8\xd0\xf8\xce\xc1\x81\xd3\xe2\xd7$\x97\x9b1'
35764,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627252525000000,5.076302,None,b'k\xfb\x9d\x0e\xa7V\x97\x0b',b'\xa5r\xab\x04\xfc\xe1ZC\xbb\xe2\xa6\xd5\xcb\xc2\x17E'
47509,"[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627278080000000,4.773562,None,b'P\x97jq\x13\xc1\xe1\xfe',b'8\xb1\xfd\xf8\xd0\xf8\xce\xc1\x81\xd3\xe2\xd7$\x97\x9b1'


In [65]:
%%markdown

#### Let's bring back the extracted exemplar columns:

Now note how one `metric` data point can get more than one exemplar, within different `traces`. (Hint: look at repeating indexes in the left-most column):


#### Let's bring back the extracted exemplar columns:

Now note how one `metric` data point can get more than one exemplar, within different `traces`. (Hint: look at repeating indexes in the left-most column):


In [66]:
exemplars_df=pd.concat([exemplars_df, exemplars_df['exemplars'].apply(pd.Series)], axis=1)

exemplars_df.head(10)

,scope_name,name,description,unit,type,start_time_unix_nano,time_unix_nano,exemplars,filtered_attributes,time_unix_nano,as_double,as_int,span_id,trace_id
1616,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723626990691310448,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723626941986000000,240.269815,None,"b'\xf0\x9fc\xf6\xda""\x1c\x15'",b'\xa64[\x94<\xeb\x88\xb70!\xb3\xb2z\xd2U\xa2'
3506,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627050690892769,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627036217000000,3.029536,None,b'\xd0[&7\x05O$\xc4',"b'c\xab\xb1\xa3""\xec\xa1koN\x93C\x91\x05\xad\x89'"
3506,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627050690892769,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627016538000000,28.312800,None,b'\xe7\x92I\x99\x97gr\xc1',b'\x94{\xa8\xe3?\xea\xc2~\xeeK\xf8\x96\x91+J\xd0'
4449,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627110690872218,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627096908000000,1.966614,None,b'\x07\xa5\x1b\xc8\xa9l\x88m',b'Z\x04\xd7\xc23d3b\x96\xc8eC7\xcc\x8d\x85'
5258,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627110690872218,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627096908000000,1.966614,None,b'\x07\xa5\x1b\xc8\xa9l\x88m',b'Z\x04\xd7\xc23d3b\x96\xc8eC7\xcc\x8d\x85'
33851,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627170691991446,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627133368000000,2.024193,None,b'E\xb6&\xf1\xc7\xe6Z0',b'\xadx\x15?\x96\x1b\xae\x0c2f\xab\x879.\x14\xdc'
34807,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627230690873998,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627174932000000,2.099705,None,b'\x81s\xbeYI\xef5\x01',b'x\xdd\xe8\xe6\xf3\x8f\xe7\xbc\xcf`\xd5\xbea\x00\x10\x8e'
35764,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627290690861235,"{'filtered_attributes': [{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44'...","[{'key': 'network.peer.address', 'value': {'string_value': '10.244.1.44', 'bool_value': None, 'i...",1723627278080000000,4.773562,None,b'P\x97jq\x13\xc1\xe1\xfe',b'8\xb1\xfd\xf8\xd0\xf8\xce\xc1\x81\xd3\xe2\xd7$\x97\x9b1'
35764,io.opentelemetry.grpc-1.6,rpc.server.duration,The duration of an inbound RPC invocation,ms,HISTOGRAM,1723626930684586610,1723627290690861235

In [67]:
%%markdown

#### Now, let's save the pre-processed metrics data as we will need it later:

***Note:*** OpenTelemetry data packets may have been retransmitted (due to network errors for example). Retransmitted data may be present more than
once in the output Parquet files. For this reason - we'll also deduplicate our metrics before saving them.


#### Now, let's save the pre-processed metrics data as we will need it later:

***Note:*** OpenTelemetry data packets may have been retransmitted (due to network errors for example). Retransmitted data may be present more than
once in the output Parquet files. For this reason - we'll also deduplicate our metrics before saving them.


In [68]:
# Convert data types to native representation and save to a pre-processed file for later use
ext_df['a_http.status_code']=pd.to_numeric(ext_df['a_http.status_code'], errors='coerce')
ext_df=ext_df.astype({'a_type': 'string'})
ext_df=ext_df.convert_dtypes()

In [69]:
# deduplicate based on resources, scopes, attribtes, etc...
dedup_cols=[c for c in ext_df.columns if c not in [
    'gauge_int', 'gauge_double',
    'sum_int', 'sum_double',
    'histogram_bucket_counts', 'histogram_explicit_bounds']
]

ext_df = ext_df.drop_duplicates(subset=dedup_cols)

In [70]:
ext_df.to_parquet('otel-demo-app/metrics.parquet', engine='pyarrow')

In [71]:
%%markdown

> ### ***To dive deeper*** into metrics data switch now to the [metrics notebook](metrics.ipynb)


> ### ***To dive deeper*** into metrics data switch now to the [metrics notebook](metrics.ipynb)


In [72]:
%%markdown

---

# Traces signal

The next OpenTelemetry signal type in this demo is the [traces signal.](https://opentelemetry.io/docs/concepts/signals/traces/)

This signal 'traces' the path through your app's code as it gets executed in order to fulfill a particular task, like, for example - to handle an incoming
client request.

A `trace_id` is assigned to each trace. Within a `trace`, `spans` (identified by `span_id`, `name` and `kind`) are opened and closed (at 
`start_time_unix_nano` and `end_time_unix_nano` respectively) to wrap around key procedures in your code - like around a database query or a REST request to another 
microservice.

`Spans` can also be nested and when so - the parent `span_id` will be recorded in `parent_span_id`. Another way of linking `spans` is through the `links` column.
During the time your app spends within a `span` it can record key `events` as they occur. If the procedure inside a `span` fails completely - the `span` 
can be closed with an error marked in the `status` column.

As with the above signal types - `traces` contain `resource`, `scope` and `span` meta-data. See the `logs` examples above for more details on `resources`, 
`scopes` and `attributes`.

When the [Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) receives `traces` it extracts each each 
`span` and saves it as a single row in the output Parquet file. In doing so it also adds a few columns to help with more complex data processing. These 
columns are not part of the original OpenTelemetry format:
- `batch_timestamp`
- `batch_UUID`
- `seq_no`
- `is_valid`
- `error_message`

Now, let's get to loading `traces` data from Parquet files and exploring it. Similarly to the above examples, we'll also extract the meta-data:


---

# Traces signal

The next OpenTelemetry signal type in this demo is the [traces signal.](https://opentelemetry.io/docs/concepts/signals/traces/)

This signal 'traces' the path through your app's code as it gets executed in order to fulfill a particular task, like, for example - to handle an incoming
client request.

A `trace_id` is assigned to each trace. Within a `trace`, `spans` (identified by `span_id`, `name` and `kind`) are opened and closed (at 
`start_time_unix_nano` and `end_time_unix_nano` respectively) to wrap around key procedures in your code - like around a database query or a REST request to another 
microservice.

`Spans` can also be nested and when so - the parent `span_id` will be recorded in `parent_span_id`. Another way of linking `spans` is through the `links` column.
During the time your app spends within a `span` it can record key `events` as they occur. If the procedure inside a `span` fails completely - the `span` 
can be closed with an error marked in the `status` column.

As with the above signal types - `traces` contain `resource`, `scope` and `span` meta-data. See the `logs` examples above for more details on `resources`, 
`scopes` and `attributes`.

When the [Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) receives `traces` it extracts each each 
`span` and saves it as a single row in the output Parquet file. In doing so it also adds a few columns to help with more complex data processing. These 
columns are not part of the original OpenTelemetry format:
- `batch_timestamp`
- `batch_UUID`
- `seq_no`
- `is_valid`
- `error_message`

Now, let's get to loading `traces` data from Parquet files and exploring it. Similarly to the above examples, we'll also extract the meta-data:


In [73]:
# Replace the raw_df DataFrame with parquet data from a given directory (note: it should only contain traces parquets):
raw_df = pd.read_parquet('otel-demo-app/traces_raw/', engine='pyarrow')
raw_df.head(25).fillna('')

,batch_timestamp,batch_UUID,seq_no,resource_attributes,resource_dropped_attributes_count,resource_schema_url,scope_name,scope_version,scope_attributes,scope_dropped_attributes_count,trace_id,span_id,trace_state,parent_span_id,flags,name,kind,start_time_unix_nano,end_time_unix_nano,attributes,dropped_attributes_count,events,dropped_events_count,links,dropped_links_count,status,span_schema_url,is_valid,error_message
0,1723626938228,24c7d982-037f-4bd3-955f-870d2a2661dd,0,"[{'key': 'service.name', 'value': {'string_value': 'frontend', 'bool_value': None, 'int_value': ...",0,,@opentelemetry/instrumentation-http,0.52.1,[],0,b'\xaa\x12\x15\xbe\xc1d\x16\xc5\x12\x18\xc4\xf2\xf8\xbd\xc0W',b'|\xc8\xb1\\\t\xda \xa4',,b'',0,GET,b'SPAN_KIND_CLIENT',1723626932850000000,1723626933041738697,"[{'key': 'http.url', 'value': {'string_value': 'http://metadata.google.internal./computeMetadata...",0,"[{'time_unix_nano': 1723626933041528195, 'name': 'exception', 'attributes': [{'key': 'exception....",0,[],0,"{'message': 'getaddrinfo ENOTFOUND metadata.google.internal.', 'code': b'STATUS_CODE_ERROR'}",,True,
1,1723626938228,24c7d982-037f-4bd3-955f-870d2a2661dd,1,"[{'key': 'service.name', 'value': {'string_value': 'frontend', 'bool_value': None, 'int_value': ...",0,,@opentelemetry/instrumentation-http,0.52.1,[],0,b'\xc0\xdeJd\xb9\xcdP\xab\x83\xa8{\xf5\x8b\xcf\rc',"b'\xde$\xf2i,\xc3\x16\x14'",,b'',0,GET,b'SPAN_KIND_CLIENT',1723626933050000000,1723626933446602174,"[{'key': 'http.url', 'value': {'string_value': 'https://kubernetes.default.svc/api/v1/namespaces...",0,[],0,[],0,"{'message': '', 'code': b'STATUS_CODE_ERROR'}",,True,
2,1723626938228,24c7d982-037f-4bd3-955f-870d2a2661dd,2,"[{'key': 'service.name', 'value': {'string_value': 'frontend', 'bool_value': None, 'int_value': ...",0,,@opentelemetry/instrumentation-http,0.52.1,[],0,"b""\x943\xcdBt\x88LW\xf5\xffB*'\x1b\x07L""",b'\xfev\x96\xde\xcb.\xc2\xed',,b'',0,GET,b'SPAN_KIND_CLIENT',1723626932846000000,1723626935855233118,"[{'key': 'http.url', 'value': {'string_value': 'http://169.254.169.254/computeMetadata/v1/instan...",0,"[{'time_unix_nano': 1723626935855224099, 'name': 'exception', 'attributes': [{'key': 'exception....",0,[],0,"{'message': 'socket hang up', 'code': b'STATUS_CODE_ERROR'}",,True,
3,1723626939175,e8aed589-1370-4f70-857a-34b3e087a135,0,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.instrumentation.requests,0.44b0,[],0,b'\xb7\xd9!V\xacL\x8a\xc7\x12\xae\x97\x90N\xc2\xba\xc8',b'\xd0)\x90\x8d\x9c\xc1\xdfA',,b'',0,GET,b'SPAN_KIND_CLIENT',1723626935105375523,1723626935119673381,"[{'key': 'http.method', 'value': {'string_value': 'GET', 'bool_value': None, 'int_value': None, ...",0,"[{'time_unix_nano': 1723626935119621929, 'name': 'exception', 'attributes': [{'key': 'exception....",0,[],0,"{'message': 'ConnectionError: HTTPConnectionPool(host='opentelemetry-demo-frontendproxy', port=8...",,True,
4,1723626939175,e8aed589-1370-4f70-857a-34b3e087a135,1,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.instrumentation.requests,0.44b0,[],0,b'\t1\xa7\xa6\x14\x9d\x05\xaf5\xd2\xe8\xca\xbd\xf3\xdf~',b'\xeb\xfdn\xbf\x8a\xcc\xa6o',,b'',0,GET,b'SPAN_KIND_CLIENT',1723626937106148978,1723626937109171178,"[{'key': 'http.method', 'value': {'string_value': 'GET', 'bool_value': None, 'int_value': None, ...",0,[],0,[],0,"{'message': '', 'code': b'STATUS_CODE_ERROR'}",,True,
5,1723626939175,e8aed589-1370-4f70-857a-34b3e087a135,2,"[{'key': 'telemetry.sdk.language', 'value': {'string_value': 'python', 'bool_value': None, 'int_...",0,,opentelemetry.instrumentation.requests,0.44b0,[],0,b'\xfa\x06\x184\xf9>\xfe\x1aZ\xcf\xfbT#\xe9\x89\xae',b'y\xb8\x10\x0cn\xb2cF',,b'',0,GET,b'SPAN_KIND_CLIENT',1723626937109761669,1723626937111121307,"[{'key': 'http.method', 'value': {'string_value': 'GET', 'bool_value': None, 'int_value': None, ...",0,[],0,[],0,"{'message': '', 'code': b'STATUS_C

In [74]:
resource_attrs_df=raw_df[['resource_attributes']].apply(otel_attrs, axis=1).add_prefix('ra_')
scope_attrs_df=raw_df[['scope_attributes']].apply(otel_attrs, axis=1).add_prefix('sa_')
attrs_df=raw_df[['attributes']].apply(otel_attrs, axis=1).add_prefix('a_')

In [75]:
%%markdown

#### Resource attributes follow the same concepts as other signals:


#### Resource attributes follow the same concepts as other signals:


In [76]:
resource_attrs_df.head(25).fillna('')

,ra_container.id,ra_host.arch,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.name,ra_os.type,ra_os.version,ra_process.command,ra_process.command_args,ra_process.command_line,ra_process.executable.name,ra_process.executable.path,ra_process.owner,ra_process.pid,ra_process.runtime.description,ra_process.runtime.name,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.auto.version,ra_telemetry.distro.name,ra_telemetry.distro.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
0,b24532380021e4d85ee7b3b2479d5d1d37698ad0bf93923b08078b146737e0f1,amd64,opentelemetry-demo-frontend-7fcb8d57f8-45frx,opentelemetry-demo-frontend,otel-demo-parquet,minikube,10.244.1.44,opentelemetry-demo-frontend-7fcb8d57f8-45frx,2024-08-14T09:15:29Z,e28a7d57-f1c3-4e59-bdb6-aae7c5c19b8e,,,linux,5.14.0-427.24.1.el9_4.x86_64,/app/server.js,ignoring attribute list value,,node,/usr/local/bin/node,nextjs,16.0,Node.js,nodejs,20.16.0,e28a7d57-f1c3-4e59-bdb6-aae7c5c19b8e,frontend,opentelemetry-demo-parquet,1.11.1,,,,nodejs,opentelemetry,1.25.1
1,b24532380021e4d85ee7b3b2479d5d1d37698ad0bf93923b08078b146737e0f1,amd64,opentelemetry-demo-frontend-7fcb8d57f8-45frx,opentelemetry-demo-frontend,otel-demo-parquet,minikube,10.244.1.44,opentelemetry-demo-frontend-7fcb8d57f8-45frx,2024-08-14T09:15:29Z,e28a7d57-f1c3-4e59-bdb6-aae7c5c19b8e,,,linux,5.14.0-427.24.1.el9_4.x86_64,/app/server.js,ignoring attribute list value,,node,/usr/local/bin/node,nextjs,16.0,Node.js,nodejs,20.16.0,e28a7d57-f1c3-4e59-bdb6-aae7c5c19b8e,frontend,opentelemetry-demo-parquet,1.11.1,,,,nodejs,opentelemetry,1.25.1
2,b24532380021e4d85ee7b3b2479d5d1d37698ad0bf93923b08078b146737e0f1,amd64,opentelemetry-demo-frontend-7fcb8d57f8-45frx,opentelemetry-demo-frontend,otel-demo-parquet,minikube,10.244.1.44,opentelemetry-demo-frontend-7fcb8d57f8-45frx,2024-08-14T09:15:29Z,e28a7d57-f1c3-4e59-bdb6-aae7c5c19b8e,,,linux,5.14.0-427.24.1.el9_4.x86_64,/app/server.js,ignoring attribute list value,,node,/usr/local/bin/node,nextjs,16.0,Node.js,nodejs,20.16.0,e28a7d57-f1c3-4e59-bdb6-aae7c5c19b8e,frontend,opentelemetry-demo-parquet,1.11.1,,,,nodejs,opentelemetry,1.25.1
3,,,,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,,,,,,,,,,,,,,,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,,,,python,opentelemetry,1.23.0
4,,,,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,,,,,,,,,,,,,,,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,,,,python,opentelemetry,1.23.0
5,,,,opentelemetry-demo-loadgenerator,otel-demo-parquet,minikube,10.244.1.48,opentelemetry-demo-loadgenerator-76f7fc9689-jsvlv,2024-08-14T09:15:29Z,c8245f82-76f9-409b-848b-1f322ab0558c,,,,,,,,,,,,,,,c8245f82-76f9-409b-848b-1f322ab0558c,loadgenerator,opentelemetry-demo-parquet,1.11.1,,,,python,opentelemetry,1.23.0
6,,amd64,opentelemetry-demo-paymentservice-74484df7d4-nfcnj,opentelemetry-demo-paymentservice,otel-demo-parquet,minikube,10.244.1.49,opentelemetry-demo-paymentservice-74484df7d4-nfcnj,2024-08-14T09:15:30Z,9e38f35f-7705-443d-9b08-54bcb6bd0dfc,,,linux,5.14.0-427.24.1.el9_4.x86_64,/usr/src/app/index.js,ignoring attribute list value,,node,/usr/local/bin/node,node,16.0,Node.js,nodejs,21.7.3,9e38f35f-7705-443d-9b08-54bcb6bd0dfc,paymentservice,opentelemetry-demo-parquet,1.11.1,,,,nodejs,opentelemetry,1.24.1
7,,amd64,opentelemetry-demo-paymentservice-74484df7d4-nfcnj,opentelemetry-demo-paymentservice,otel-demo-parquet,minikube,10.244.1.49,opentelemetry-demo-paymentservice-74484df7d4-nfcnj,2024-08-14T09:15:30Z,9e38f35f-7705-443d-9b08-54

In [77]:
%%markdown

#### ...and we see some familiar and some new scopes:


#### ...and we see some familiar and some new scopes:


In [78]:
raw_df[['scope_name']].drop_duplicates()

,scope_name
0,@opentelemetry/instrumentation-http
3,opentelemetry.instrumentation.requests
6,@opentelemetry/instrumentation-dns
9,@opentelemetry/instrumentation-net
15,jsonEvaluator
16,
46,currencyservice
91,@opentelemetry/instrumentation-document-load
107,@opentelemetry/instrumentation-fetch
119,opentelemetry.instrumentation.grpc


In [79]:
%%markdown

#### ...and a variety of span meta-data:

***Hint:*** Use the horizontal scrollbar at the bottom


#### ...and a variety of span meta-data:

***Hint:*** Use the horizontal scrollbar at the bottom


In [80]:
attrs_df.head(25).fillna('')

,a_app.ads.ad_request_type,a_app.ads.ad_response_type,a_app.ads.category,a_app.ads.contextKeys,a_app.ads.contextKeys.count,a_app.ads.count,a_app.cart.items.count,a_app.currency.conversion.from,a_app.currency.conversion.to,a_app.email.recipient,a_app.filtered_products.count,a_app.filtered_products.list,a_app.order.amount,a_app.order.id,a_app.order.items.count,a_app.payment.amount,a_app.payment.card_type,a_app.payment.card_valid,a_app.payment.charged,a_app.product.id,a_app.product.name,a_app.product.quantity,a_app.products.count,a_app.products_recommended.count,a_app.quote.cost.total,a_app.quote.items.count,a_app.recommendation.cache_enabled,a_app.shipping.amount,a_app.shipping.cost.total,a_app.shipping.items.count,a_app.shipping.tracking.id,a_app.shipping.zip_code,a_app.synthetic_request,a_app.user.currency,a_app.user.id,a_busy_ns,a_canceled,a_code.filepath,a_code.function,a_code.lineno,a_code.namespace,a_component,a_db.redis.database_index,a_db.redis.flags,a_db.statement,a_db.system,a_dns.error_message,a_dns.error_name,a_downstream_cluster,a_error,a_error.reason,a_error.type,a_feature_flag.change_count,a_feature_flag.count,a_feature_flag.key,a_feature_flag.provider_name,a_feature_flag.source,a_feature_flag.sync_type,a_feature_flag.variant,a_grpc.error_message,a_grpc.error_name,a_grpc.method,a_grpc.status_code,a_guid:x-request-id,a_http.error_message,a_http.error_name,a_http.flavor,a_http.host,a_http.method,a_http.protocol,a_http.request.body.size,a_http.request.method,a_http.request_content_length,a_http.request_content_length_uncompressed,a_http.response.body.size,a_http.response.status_code,a_http.response_content_length,a_http.response_content_length_uncompressed,a_http.route,a_http.scheme,a_http.status_code,a_http.status_text,a_http.target,a_http.url,a_http.user_agent,a_idle_ns,a_messaging.client_id,a_messaging.destination.name,a_messaging.destination.partition.id,a_messaging.kafka.consumer.group,a_messaging.kafka.destination.partition,a_messaging.kafka.message.offset,a_messaging.kafka.producer.duration_ms,a_messaging.kafka.producer.success,a_messaging.message.body.size,a_messaging.operation,a_messaging.system,a_net.host.ip,a_net.host.name,a_net.host.port,a_net.peer.ip,a_net.peer.name,a_net.peer.port,a_net.sock.peer.addr,a_net.sock.peer.port,a_net.transport,a_network.peer.address,a_network.peer.port,a_network.protocol.version,a_network.transport,a_network.type,a_next.route,a_next.rsc,a_next.span_name,a_next.span_type,a_node_id,a_peer.address,a_peer.ipv4,a_peer.service,a_request_size,a_response_flags,a_response_size,a_rpc.connect_rpc.error_code,a_rpc.grpc.status_code,a_rpc.method,a_rpc.service,a_rpc.system,a_rpc.user_agent,a_server.address,a_server.port,a_session.id,a_sinatra.template_name,a_thread.id,a_thread.name,a_tls.alpnProtocol,a_tls.authorized,a_tls.certificate.fingerprint,a_tls.certificate.serialNumber,a_tls.certificate.validFrom,a_tls.certificate.validTo,a_tls.cipher.name,a_tls.cipher.version,a_tls.protocol,a_upstream_address,a_upstream_cluster,a_upstream_cluster.name,a_url.full,a_url.path,a_url.scheme,a_user_agent,a_user_agent.original,a_zone
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,getaddrinfo ENOTFOUND metadata.google.internal.,Error,,metadata.google.internal.:80,GET,,,,,,,,,,,,,,/computeMetadata/v1/instance,http://metadata.google.internal./computeMetadata/v1/instance,,,,,,,,,,,,,,,,,,metadata.google.internal.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1.1,kubernetes.default.svc:443,GET,,,,,,,,,355.0,,,403.0,FORBIDDEN,/api/v1/namespaces/kube-system/configmaps/aws-auth,https://kubernetes.default.svc/api/v1/namespaces/kube-system/configmaps/aws-auth,,,,,,,,,,,,,,,,,10.96.0.1,kubernetes.default.svc,443.0,,,ip_tcp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,socket hang up,Error,,169.254.169.254:80,GET,,,,,,,,,,,,,,/computeMetadata/v1/instance,http://1

In [81]:
%%markdown

# Spans

The `traces` signal records a few useful parameters about each span:
- a `name` that typically contains a way of identifying what operation is performed within it (like a HTTP request, a method in your code or similar)
- a `kind` that labels a span as `CLIENT`, `SERVER`, `PRODUCER`, `CONSUMER` or `INTERNAL`

  These values can help you identify which ***side*** the span is running on. For example, a span around a HTTP GET method to a particular URL can 
  be opened by the HTTP ***client*** and also by the HTTP ***server*** handling it. Typically, in this case the ***client-side*** span will be marked 
  as `SPAN_KIND_CLIENT` and the ***server-side*** as `SPAN_KIND_SERVER`.
  
  Also, thanks to 
  [OpenTelemetry Context Propagation](https://opentelemetry.io/docs/concepts/context-propagation/)
  the 'server-side' span can be correlated to its corresponding 'client-side' and have its `parent_span_id` column set.

  For more, go to [OpenTelemetry Span Kinds.](https://opentelemetry.io/docs/concepts/signals/traces/#span-kind)



# Spans

The `traces` signal records a few useful parameters about each span:
- a `name` that typically contains a way of identifying what operation is performed within it (like a HTTP request, a method in your code or similar)
- a `kind` that labels a span as `CLIENT`, `SERVER`, `PRODUCER`, `CONSUMER` or `INTERNAL`

  These values can help you identify which ***side*** the span is running on. For example, a span around a HTTP GET method to a particular URL can 
  be opened by the HTTP ***client*** and also by the HTTP ***server*** handling it. Typically, in this case the ***client-side*** span will be marked 
  as `SPAN_KIND_CLIENT` and the ***server-side*** as `SPAN_KIND_SERVER`.
  
  Also, thanks to 
  [OpenTelemetry Context Propagation](https://opentelemetry.io/docs/concepts/context-propagation/)
  the 'server-side' span can be correlated to its corresponding 'client-side' and have its `parent_span_id` column set.

  For more, go to [OpenTelemetry Span Kinds.](https://opentelemetry.io/docs/concepts/signals/traces/#span-kind)


In [82]:
raw_df[['name', 'kind']].drop_duplicates()

,name,kind
0,GET,b'SPAN_KIND_CLIENT'
6,dns.lookup,b'SPAN_KIND_CLIENT'
9,tcp.connect,b'SPAN_KIND_INTERNAL'
11,tls.connect,b'SPAN_KIND_INTERNAL'
15,flagSync,b'SPAN_KIND_INTERNAL'
16,router frontend egress,b'SPAN_KIND_CLIENT'
17,ingress,b'SPAN_KIND_SERVER'
43,router flagservice egress,b'SPAN_KIND_CLIENT'
46,CurrencyService/GetSupportedCurrencies,b'SPAN_KIND_SERVER'
52,router imageprovider egress,b'SPAN_KIND_CLIENT'


In [83]:
%%markdown

## Span status

As often with running code - particular operations may fail. When that happens - the span will record an error in its `status.code` field
and potentially an error message inside `status.message`.

Span `status.code`s are `STATUS_CODE_ERROR` (when an error was encountered) or `STATUS_CODE_UNSET`/`STATUS_CODE_OK` when the code completed successfully.
`UNSET` and `OK` both mean the same, the difference being only that to get `OK` your code must specifically mark the span as 
finished without an error. If there's no error detected during the execution of your code, and your code does not close its span with `OK` - the 
default status `UNSET` will be reported by OpenTelemetry.


## Span status

As often with running code - particular operations may fail. When that happens - the span will record an error in its `status.code` field
and potentially an error message inside `status.message`.

Span `status.code`s are `STATUS_CODE_ERROR` (when an error was encountered) or `STATUS_CODE_UNSET`/`STATUS_CODE_OK` when the code completed successfully.
`UNSET` and `OK` both mean the same, the difference being only that to get `OK` your code must specifically mark the span as 
finished without an error. If there's no error detected during the execution of your code, and your code does not close its span with `OK` - the 
default status `UNSET` will be reported by OpenTelemetry.


In [84]:
# note: the status column is an object with fields inside, with the following we'll turn them into separate columns:
pd.DataFrame(raw_df['status'].head(10).apply(pd.Series))

,message,code
0,getaddrinfo ENOTFOUND metadata.google.internal.,b'STATUS_CODE_ERROR'
1,,b'STATUS_CODE_ERROR'
2,socket hang up,b'STATUS_CODE_ERROR'
3,"ConnectionError: HTTPConnectionPool(host='opentelemetry-demo-frontendproxy', port=8080): Max ret...",b'STATUS_CODE_ERROR'
4,,b'STATUS_CODE_ERROR'
5,,b'STATUS_CODE_ERROR'
6,getaddrinfo ENOTFOUND metadata.google.internal.,b'STATUS_CODE_ERROR'
7,,b'STATUS_CODE_UNSET'
8,,b'STATUS_CODE_UNSET'
9,getaddrinfo ENOTFOUND metadata.google.internal.,b'STATUS_CODE_ERROR'


In [85]:
%%markdown

## Events within a Span

During a span sometimes it might be useful to record key events occurring. Your code can use the `logs` signal and then you can correlate, but also, you
can use the OpenTelemetry APIs to add `events` to your spans. These will be added together in the `events` column, and will contain `time_unix_nano` 
of each event, a `name` for it and optionally some `attributes`.

`Events` are particularly useful if you'd like to record some strucutred data about them, rather than text that needs to be parsed later:


## Events within a Span

During a span sometimes it might be useful to record key events occurring. Your code can use the `logs` signal and then you can correlate, but also, you
can use the OpenTelemetry APIs to add `events` to your spans. These will be added together in the `events` column, and will contain `time_unix_nano` 
of each event, a `name` for it and optionally some `attributes`.

`Events` are particularly useful if you'd like to record some strucutred data about them, rather than text that needs to be parsed later:


In [86]:
# the events column contains an array of objects - lets 'explode' it to rows first, and then extract the columns:
pd.DataFrame(raw_df[raw_df['events'].apply(lambda e: e.size) > 0].explode('events')['events'].head(20).apply(pd.Series))

,time_unix_nano,name,attributes,dropped_attributes_count
0,1723626933041528195,exception,"[{'key': 'exception.type', 'value': {'string_value': 'ENOTFOUND', 'bool_value': None, 'int_value...",0
2,1723626935855224099,exception,"[{'key': 'exception.type', 'value': {'string_value': 'ECONNRESET', 'bool_value': None, 'int_valu...",0
3,1723626935119621929,exception,"[{'key': 'exception.type', 'value': {'string_value': 'ConnectionError', 'bool_value': None, 'int...",0
13,1723626934356311987,exception,"[{'key': 'exception.type', 'value': {'string_value': 'ENOTFOUND', 'bool_value': None, 'int_value...",0
14,1723626937354592963,exception,"[{'key': 'exception.type', 'value': {'string_value': 'ECONNRESET', 'bool_value': None, 'int_valu...",0
46,1723626941360601974,Processing supported currencies request,[],0
46,1723626941360616841,"Currencies fetched, response sent back",[],0
62,1723626941449855516,Processing currency conversion request,[],0
62,1723626941452815961,"Conversion successful, response sent back",[],0
63,1723626941449917434,Processing currency conversion request,[],0


In [87]:
%%markdown

# Let's do a bit of data pre-processing:


# Let's do a bit of data pre-processing:


In [88]:
# get a base DataFrame with select columns only
base_df=raw_df.loc[raw_df['is_valid'] == True][
    [
        'scope_name',
        'scope_version',
        'trace_id',
        'span_id',
        'trace_state',
        'parent_span_id',
        'flags',
        'name',
        'kind',
        'start_time_unix_nano',
        'end_time_unix_nano',
    ]
]
# also extract a few nested fields
base_df['status.message']=raw_df.loc[raw_df['is_valid'] == True]['status'].apply(lambda s: s.get('message'))
base_df['status.code']=raw_df.loc[raw_df['is_valid'] == True]['status'].apply(lambda s: s.get('code'))
base_df['events']=raw_df.loc[raw_df['is_valid'] == True]['events']
base_df['links']=raw_df.loc[raw_df['is_valid'] == True]['links']

In [89]:
# 'join' with extracted attributes:
ext_df=base_df.join(attrs_df).join(resource_attrs_df).join(scope_attrs_df)
ext_df.head(25).fillna('')

,scope_name,scope_version,trace_id,span_id,trace_state,parent_span_id,flags,name,kind,start_time_unix_nano,end_time_unix_nano,status.message,status.code,events,links,a_app.ads.ad_request_type,a_app.ads.ad_response_type,a_app.ads.category,a_app.ads.contextKeys,a_app.ads.contextKeys.count,a_app.ads.count,a_app.cart.items.count,a_app.currency.conversion.from,a_app.currency.conversion.to,a_app.email.recipient,a_app.filtered_products.count,a_app.filtered_products.list,a_app.order.amount,a_app.order.id,a_app.order.items.count,a_app.payment.amount,a_app.payment.card_type,a_app.payment.card_valid,a_app.payment.charged,a_app.product.id,a_app.product.name,a_app.product.quantity,a_app.products.count,a_app.products_recommended.count,a_app.quote.cost.total,a_app.quote.items.count,a_app.recommendation.cache_enabled,a_app.shipping.amount,a_app.shipping.cost.total,a_app.shipping.items.count,a_app.shipping.tracking.id,a_app.shipping.zip_code,a_app.synthetic_request,a_app.user.currency,a_app.user.id,a_busy_ns,a_canceled,a_code.filepath,a_code.function,a_code.lineno,a_code.namespace,a_component,a_db.redis.database_index,a_db.redis.flags,a_db.statement,a_db.system,a_dns.error_message,a_dns.error_name,a_downstream_cluster,a_error,a_error.reason,a_error.type,a_feature_flag.change_count,a_feature_flag.count,a_feature_flag.key,a_feature_flag.provider_name,a_feature_flag.source,a_feature_flag.sync_type,a_feature_flag.variant,a_grpc.error_message,a_grpc.error_name,a_grpc.method,a_grpc.status_code,a_guid:x-request-id,a_http.error_message,a_http.error_name,a_http.flavor,a_http.host,a_http.method,a_http.protocol,a_http.request.body.size,a_http.request.method,a_http.request_content_length,a_http.request_content_length_uncompressed,a_http.response.body.size,a_http.response.status_code,a_http.response_content_length,a_http.response_content_length_uncompressed,a_http.route,a_http.scheme,a_http.status_code,a_http.status_text,a_http.target,a_http.url,a_http.user_agent,a_idle_ns,a_messaging.client_id,a_messaging.destination.name,a_messaging.destination.partition.id,a_messaging.kafka.consumer.group,a_messaging.kafka.destination.partition,a_messaging.kafka.message.offset,a_messaging.kafka.producer.duration_ms,a_messaging.kafka.producer.success,a_messaging.message.body.size,a_messaging.operation,a_messaging.system,a_net.host.ip,a_net.host.name,a_net.host.port,a_net.peer.ip,a_net.peer.name,a_net.peer.port,a_net.sock.peer.addr,a_net.sock.peer.port,a_net.transport,a_network.peer.address,a_network.peer.port,a_network.protocol.version,a_network.transport,a_network.type,a_next.route,a_next.rsc,a_next.span_name,a_next.span_type,a_node_id,a_peer.address,a_peer.ipv4,a_peer.service,a_request_size,a_response_flags,a_response_size,a_rpc.connect_rpc.error_code,a_rpc.grpc.status_code,a_rpc.method,a_rpc.service,a_rpc.system,a_rpc.user_agent,a_server.address,a_server.port,a_session.id,a_sinatra.template_name,a_thread.id,a_thread.name,a_tls.alpnProtocol,a_tls.authorized,a_tls.certificate.fingerprint,a_tls.certificate.serialNumber,a_tls.certificate.validFrom,a_tls.certificate.validTo,a_tls.cipher.name,a_tls.cipher.version,a_tls.protocol,a_upstream_address,a_upstream_cluster,a_upstream_cluster.name,a_url.full,a_url.path,a_url.scheme,a_user_agent,a_user_agent.original,a_zone,ra_container.id,ra_host.arch,ra_host.name,ra_k8s.deployment.name,ra_k8s.namespace.name,ra_k8s.node.name,ra_k8s.pod.ip,ra_k8s.pod.name,ra_k8s.pod.start_time,ra_k8s.pod.uid,ra_os.description,ra_os.name,ra_os.type,ra_os.version,ra_process.command,ra_process.command_args,ra_process.command_line,ra_process.executable.name,ra_process.executable.path,ra_process.owner,ra_process.pid,ra_process.runtime.description,ra_process.runtime.name,ra_process.runtime.version,ra_service.instance.id,ra_service.name,ra_service.namespace,ra_service.version,ra_telemetry.auto.version,ra_telemetry.distro.name,ra_telemetry.distro.version,ra_telemetry.sdk.language,ra_telemetry.sdk.name,ra_telemetry.sdk.version
0,@opentelemetry/instrumentation

In [90]:
%%markdown

# Exploring traces

Let's find the same `trace_id` that we used in the `logs` correlation example above and take a deeper look at what the [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) was doing:

> ***Note:*** Look at the `parent_span_id` values and the rows they point to to see how spans get subdivided


# Exploring traces

Let's find the same `trace_id` that we used in the `logs` correlation example above and take a deeper look at what the [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) was doing:

> ***Note:*** Look at the `parent_span_id` values and the rows they point to to see how spans get subdivided


In [91]:
trace = ext_df[ext_df['trace_id'] == trace_id].sort_values('start_time_unix_nano')

trace[
    [
        # span emitter:
        'ra_service.name', 'ra_telemetry.sdk.language', 'ra_process.command', 'ra_process.executable.path', 'scope_name',

        # the span itself:
        'name', 'span_id', 'parent_span_id', 'kind', 'status.code', 'start_time_unix_nano', 'end_time_unix_nano',
        
        # some custom span meta-data emitted by the demo Astronomy webshop microservices:
        'a_app.cart.items.count', 'a_app.currency.conversion.from', 'a_app.currency.conversion.to', 'a_app.email.recipient',
        'a_app.order.amount', 'a_app.order.id', 'a_app.order.items.count', 'a_app.payment.amount', 'a_app.payment.card_type',
        'a_app.payment.card_valid', 'a_app.payment.charged', 'a_app.product.id', 'a_app.product.name', 'a_app.product.quantity',
        'a_app.quote.cost.total', 'a_app.quote.items.count', 'a_app.shipping.amount', 'a_app.shipping.cost.total',
        'a_app.shipping.items.count', 'a_app.shipping.tracking.id', 'a_app.shipping.zip_code', 'a_app.user.currency', 'a_app.user.id',

        # feature flags usage attributes:
        'a_feature_flag.key', 'a_feature_flag.provider_name', 'a_feature_flag.variant',

        # spans emitted by a redis client lib:
        'a_db.redis.database_index', 'a_db.redis.flags', 'a_db.statement', 'a_db.system',

        # Apache Kafka span attributes:
        'a_messaging.client_id', 'a_messaging.destination.name', 'a_messaging.destination.partition.id', 'a_messaging.kafka.consumer.group',
        'a_messaging.operation', 'a_messaging.system',
        
        # gRPC-related attributes:
        'a_grpc.method', 'a_grpc.status_code', 'a_rpc.method', 'a_rpc.service', 'a_rpc.system',

        # HTTP-related attributes:
        'a_http.method', 'a_http.request.method', 'a_http.route', 'a_http.target', 'a_http.url', 'a_url.full', 'a_url.path',
        'a_http.response.status_code', 'a_http.status_code', 'a_net.peer.name','a_http.host',
    ]
].fillna('')


,ra_service.name,ra_telemetry.sdk.language,ra_process.command,ra_process.executable.path,scope_name,name,span_id,parent_span_id,kind,status.code,start_time_unix_nano,end_time_unix_nano,a_app.cart.items.count,a_app.currency.conversion.from,a_app.currency.conversion.to,a_app.email.recipient,a_app.order.amount,a_app.order.id,a_app.order.items.count,a_app.payment.amount,a_app.payment.card_type,a_app.payment.card_valid,a_app.payment.charged,a_app.product.id,a_app.product.name,a_app.product.quantity,a_app.quote.cost.total,a_app.quote.items.count,a_app.shipping.amount,a_app.shipping.cost.total,a_app.shipping.items.count,a_app.shipping.tracking.id,a_app.shipping.zip_code,a_app.user.currency,a_app.user.id,a_feature_flag.key,a_feature_flag.provider_name,a_feature_flag.variant,a_db.redis.database_index,a_db.redis.flags,a_db.statement,a_db.system,a_messaging.client_id,a_messaging.destination.name,a_messaging.destination.partition.id,a_messaging.kafka.consumer.group,a_messaging.operation,a_messaging.system,a_grpc.method,a_grpc.status_code,a_rpc.method,a_rpc.service,a_rpc.system,a_http.method,a_http.request.method,a_http.route,a_http.target,a_http.url,a_url.full,a_url.path,a_http.response.status_code,a_http.status_code,a_net.peer.name,a_http.host
6406,loadgenerator,python,,,opentelemetry.instrumentation.requests,POST,b'\xb7W\x1f\x9d+~#R',b'',b'SPAN_KIND_CLIENT',b'STATUS_CODE_ERROR',1723627134056954401,1723627149059298090,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,POST,,,,http://opentelemetry-demo-frontendproxy:8080/api/checkout,,,,504.0,,
6768,frontend,nodejs,/app/server.js,/usr/local/bin/node,@opentelemetry/instrumentation-http,POST,b'\x02J\xa0r\xe6uLD',b'm\xcd\xb1\xa3\xf9nf[',b'SPAN_KIND_SERVER',b'STATUS_CODE_UNSET',1723627134057000000,1723627149058150531,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,POST,,,/api/checkout,http://opentelemetry-demo-frontendproxy:8080/api/checkout,,,,200.0,,opentelemetry-demo-frontendproxy:8080
21080,frontend,nodejs,/app/server.js,/usr/local/bin/node,next.js,POST /api/checkout,b'\xae\x04\xe8\x94\xfd\r\xedj',b'\x02J\xa0r\xe6uLD',b'SPAN_KIND_SERVER',b'STATUS_CODE_UNSET',1723627134057000000,1723627338572770300,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,POST,,,/api/checkout,,,,,200.0,,
6350,frontendproxy,,,,,ingress,b'\x14K\xab\xa5\x02\xe6\xeb\xd7',b'\xb7W\x1f\x9d+~#R',b'SPAN_KIND_SERVER',b'STATUS_CODE_UNSET',1723627134057382000,1723627149058713000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,POST,,,,http://opentelemetry-demo-frontendproxy:8080/api/checkout,,,,504,,
6349,frontendproxy,,,,,router frontend egress,b'm\xcd\xb1\xa3\xf9nf[',b'\x14K\xab\xa5\x02\xe6\xeb\xd7',b'SPAN_KIND_CLIENT',b'STATUS_CODE_UNSET',1723627134057489000,1723627149058502000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,
20269,frontend,nodejs,/app/server.js,/usr/local/bin/node,@opentelemetry/instrumentation-grpc,grpc.oteldemo.CheckoutService/PlaceOrder,b'07\xae9\xbd~\xd8v',b'\x9a\xde\xdb\x93\xb6d\t\xa3',b'SPAN_KIND_CLIENT',b'STATUS_CODE_UNSET',1723627134058000000,1723627315976950050,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,PlaceOrder,oteldemo.CheckoutService,grpc,,,,,,,,,,opentelemetry-demo-checkoutservice,
21079,frontend,nodejs,/app/server.js,/usr/local/bin/node,next.js,executing api route (pages) /api/checkout,b'\x9a\xde\xdb\x93\xb6d\t\xa3',b'\xae\x04\xe8\x94\xfd\r\xedj',b'SPAN_KIND_INTERNAL',b'STATUS_CODE_UNSET',1723627134058000000,1723627338573434952,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,200.0,,
20397,checkoutservice,go,,/usr/src/app/checkoutservice,go.opentelemetry.io/contrib/instrumentation/google.golang.org/grpc/otelgrpc,oteldemo.CheckoutService/PlaceOrder,b'y*\x1b+\xa6a\xe79',b'07\xae9\xbd~\xd8v',b'SPAN_KIND_SERVER',b'STATUS_CODE_UNSET',1723627134079549113,1723627315976423603,,,,,993.0,3a08999f-5a1e-11ef-ac87-9e6131d66c8e,2.0,,,,,,,,,,234.0,,,fea5c3a6-b231-4548-b8fb-4b5bd4d953e9,,USD,39fe30ba-5a1e-11ef-88c4-32eb2a9fa2f1,,,,,,,,,,,,,,,,PlaceOrder,oteldemo.CheckoutService,grpc,,,,,,,,,,,
4686,checkoutservice,go,,/usr/src/app/checkout

In [92]:
%%markdown

# Correlating signals

Earlier, we mentioned that `traces` are particularly useful to get the full picture (including other telemetry signals) of a running system. We also saved a few `logs` to quickly illustrate how this can be done.

We'll now 'join' those `logs` to their respective `spans` and show details about the context surrounding a given log message:


# Correlating signals

Earlier, we mentioned that `traces` are particularly useful to get the full picture (including other telemetry signals) of a running system. We also saved a few `logs` to quickly illustrate how this can be done.

We'll now 'join' those `logs` to their respective `spans` and show details about the context surrounding a given log message:


In [93]:
logs_with_trace=pd.concat([trace_logs.set_index('span_id')[['severity_text', 'body', 'a_userId']], trace.set_index('span_id')], axis=1, join='inner')
logs_with_trace[
    [
        # taken from trace_logs:
        'severity_text', 'body', 'a_userId',
        # taken from trace:
        'name', 'a_app.currency.conversion.from', 'a_app.currency.conversion.to', 'a_app.cart.items.count',
        'a_messaging.destination.name', 'a_messaging.destination.partition.id', 'a_messaging.kafka.message.offset',
        'a_grpc.method', 'a_rpc.method', 'a_rpc.service'
    ]
].fillna('')

,severity_text,body,a_userId,name,a_app.currency.conversion.from,a_app.currency.conversion.to,a_app.cart.items.count,a_messaging.destination.name,a_messaging.destination.partition.id,a_messaging.kafka.message.offset,a_grpc.method,a_rpc.method,a_rpc.service
span_id,,,,,,,,,,,,,
b'\xac\x05@P\x17\xa4\xab\xe7',INFO,Convert conversion successful,,CurrencyService/Convert,USD,USD,,,,,,Convert,oteldemo.CurrencyService
b'\x83d\xf4\xbe\x84\xb6\xf9A',INFO,Convert conversion successful,,CurrencyService/Convert,USD,USD,,,,,,Convert,oteldemo.CurrencyService
b'\x8c\xb4\xda\xcb\x12\x85\xa6f',INFO,Convert conversion successful,,CurrencyService/Convert,USD,USD,,,,,,Convert,oteldemo.CurrencyService
b'\xaa\xb6\xba-\x11-{\xe0',INFO,Calculated quote,,{closure},,,,,,,,,
b'\x04\xbe\xa8\xb3\xaa\xf4<$',Information,GetCartAsync called with userId={userId},39fe30ba-5a1e-11ef-88c4-32eb2a9fa2f1,POST /oteldemo.CartService/GetCart,,,4.0,,,,/oteldemo.CartService/GetCart,,
b'\xf7\xf1\xd2\xf1\xba\xeb4\xfc',INFO,"Consumed record with orderId: 3a08999f-5a1e-11ef-ac87-9e6131d66c8e, and updated total count to: 10",,orders process,,,,orders,0,9.0,,,
b'\xb4A-n1\x1b\\\xb1',Information,EmptyCartAsync called with userId={userId},39fe30ba-5a1e-11ef-88c4-32eb2a9fa2f1,POST /oteldemo.CartService/EmptyCart,,,,,,,/oteldemo.CartService/EmptyCart,,


In [94]:
ext_df['a_http.status_code']=pd.to_numeric(ext_df['a_http.status_code'], errors='coerce')
ext_df['a_net.host.port']=pd.to_numeric(ext_df['a_net.host.port'], errors='coerce')
ext_df['a_net.peer.port']=pd.to_numeric(ext_df['a_net.peer.port'], errors='coerce')

ext_df.convert_dtypes().to_parquet('otel-demo-app/traces.parquet', engine='pyarrow')

In [95]:
%%markdown

---

# Profiles signal

> ***Important:*** OpenTelemetry `profiles` signal is still in its early and experimental stage.
>
> Examples here will almost certainly change as development progresses.
>
> To collect the data for the following sections we used an early version of [OpenTelemetry eBPF Profiler,](https://github.com/open-telemetry/opentelemetry-ebpf-profiler) running it separately.
> This means that the DataFrames below contain data from the [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) (same demo webshop as in the examples of other signals above) and a number of other processes running on the same host.

The last signal type in this demo is the `profiles` signal and its a new extension to OpenTelemetry. The intent of performance profiling is to help you analyze the performance of your app and find code 
sections to improve.

Somewhat similar to the `traces` signal, but on the lower 'code' level, profiling measures how often each method is called and how much time a CPU spends executing that method.
As profiling data is based on **stack frames** - you also get details like where was that method called from. Unlike `traces`, `profiles` can get deeply into a running system and record calls to system libraries and kernel calls.
These allow you to understand the performance of your app including aspects like how efficiently it reads data from disks, how much it spends 'sleeping' on network I/O and other potential bottlenecks.

In the saved Parquet files, a `profile` will be measured from `start_time_unix_nano` to `end_time_unix_nano` and will be assigned an unique `profile_id`. It can also have meta-data in `profile_attributes`.

Each `profile` contains a number of **samples**, collected over a `period` of a `period_type`. Samples are collected at `time_nanos` with a possible `duration_nanos`. The **Stack frame** of each sample is saved into the `locations`
column and can be identified by its `stacktrace_id`. Additionally, **samples** can have associated `timestamps_unix_nano`, a link to a `trace_id` and `span_id`, `labels` and `attributes`. A sample will also have one or more `value` of `type`.

Also present is the `resource` and `scope` meta-data (see the `logs` example at the top for more on these).

The [OTLP Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) 'flattens' incoming signals down to a each `value` of each **sample** and adds extra columns:
- `batch_timestamp`
- `batch_UUID`
- `resource_seq_no`
- `profile_seq_no`
- `sample_seq_no`
- `value_seq_no`
- `is_valid`
- `error_message`

These columns are added to assist with more advanced analytics and are not part of the original OpenTelemetry data schema.

Now, let's load `profiles` into a DataFrame and extract meta-data:


---

# Profiles signal

> ***Important:*** OpenTelemetry `profiles` signal is still in its early and experimental stage.
>
> Examples here will almost certainly change as development progresses.
>
> To collect the data for the following sections we used an early version of [OpenTelemetry eBPF Profiler,](https://github.com/open-telemetry/opentelemetry-ebpf-profiler) running it separately.
> This means that the DataFrames below contain data from the [demo Astronomy webshop](https://github.com/mishmash-io/opentelemetry-demo-to-parquet) (same demo webshop as in the examples of other signals above) and a number of other processes running on the same host.

The last signal type in this demo is the `profiles` signal and its a new extension to OpenTelemetry. The intent of performance profiling is to help you analyze the performance of your app and find code 
sections to improve.

Somewhat similar to the `traces` signal, but on the lower 'code' level, profiling measures how often each method is called and how much time a CPU spends executing that method.
As profiling data is based on **stack frames** - you also get details like where was that method called from. Unlike `traces`, `profiles` can get deeply into a running system and record calls to system libraries and kernel calls.
These allow you to understand the performance of your app including aspects like how efficiently it reads data from disks, how much it spends 'sleeping' on network I/O and other potential bottlenecks.

In the saved Parquet files, a `profile` will be measured from `start_time_unix_nano` to `end_time_unix_nano` and will be assigned an unique `profile_id`. It can also have meta-data in `profile_attributes`.

Each `profile` contains a number of **samples**, collected over a `period` of a `period_type`. Samples are collected at `time_nanos` with a possible `duration_nanos`. The **Stack frame** of each sample is saved into the `locations`
column and can be identified by its `stacktrace_id`. Additionally, **samples** can have associated `timestamps_unix_nano`, a link to a `trace_id` and `span_id`, `labels` and `attributes`. A sample will also have one or more `value` of `type`.

Also present is the `resource` and `scope` meta-data (see the `logs` example at the top for more on these).

The [OTLP Parquet Server](https://github.com/mishmash-io/opentelemetry-server-embedded/tree/main/server-parquet) 'flattens' incoming signals down to a each `value` of each **sample** and adds extra columns:
- `batch_timestamp`
- `batch_UUID`
- `resource_seq_no`
- `profile_seq_no`
- `sample_seq_no`
- `value_seq_no`
- `is_valid`
- `error_message`

These columns are added to assist with more advanced analytics and are not part of the original OpenTelemetry data schema.

Now, let's load `profiles` into a DataFrame and extract meta-data:


In [96]:
# Read a directory of profiles parquets
raw_df = pd.read_parquet('otel-demo-app/profiles_raw/', engine='pyarrow')
# Show some raw data
raw_df.head(5).fillna('')

,batch_timestamp,batch_UUID,resource_seq_no,resource_attributes,resource_dropped_attributes_count,resource_schema_url,scope_seq_no,scope_name,scope_version,scope_attributes,scope_dropped_attributes_count,profile_schema_url,profile_seq_no,profile_id,start_time_unix_nano,end_time_unix_nano,profile_attributes,profile_dropped_attributes_count,original_payload_format,original_payload,drop_frames,keep_frames,time_nanos,duration_nanos,period_type,period,comment,default_sample_type,sample_seq_no,stacktrace_id,locations,labels,attributes,timestamps_unix_nano,trace_id,span_id,value_seq_no,value,type,is_valid,error_message
0,1724748418323,6710517f-b524-48fe-8305-4673e668f3dd,0,"[{'key': 'host.id', 'value': {'string_value': '0', 'bool_value': None, 'int_value': None, 'doubl...",0,,0,,,[],0,,0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,[],0,,b'',,,1724748417093736128,0,"{'type': 'cpu', 'unit': 'nanoseconds', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSP...",50000000,[],,0,iTnG0oMa50esbnFSMfnBIA,"[{'mapping': {'memory_start': 0, 'memory_limit': 0, 'file_offset': 0, 'filename': '', 'build_id'...",[],"[{'key': 'container.id', 'value': {'string_value': '/libpod_parent/libpod-0ba85205d552a1f876ba13...",[1724748417243725237],b'',b'',0,1,"{'type': 'samples', 'unit': 'count', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSPEC...",True,
1,1724748418323,6710517f-b524-48fe-8305-4673e668f3dd,0,"[{'key': 'host.id', 'value': {'string_value': '0', 'bool_value': None, 'int_value': None, 'doubl...",0,,0,,,[],0,,0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,[],0,,b'',,,1724748417093736128,0,"{'type': 'cpu', 'unit': 'nanoseconds', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSP...",50000000,[],,1,WYqKLHYa3mJGRunbvCksdA,"[{'mapping': {'memory_start': 118784, 'memory_limit': 712704, 'file_offset': 118784, 'filename':...",[],"[{'key': 'container.id', 'value': {'string_value': '/user.slice/user-1000.slice/user@1000.servic...",[1724748417943747682],b'',b'',0,1,"{'type': 'samples', 'unit': 'count', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSPEC...",True,
2,1724748418323,6710517f-b524-48fe-8305-4673e668f3dd,0,"[{'key': 'host.id', 'value': {'string_value': '0', 'bool_value': None, 'int_value': None, 'doubl...",0,,0,,,[],0,,0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,[],0,,b'',,,1724748417093736128,0,"{'type': 'cpu', 'unit': 'nanoseconds', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSP...",50000000,[],,2,3q0cPtu3MDx3nKTLoW1dYw,"[{'mapping': {'memory_start': 118784, 'memory_limit': 712704, 'file_offset': 118784, 'filename':...",[],"[{'key': 'container.id', 'value': {'string_value': '/user.slice/user-1000.slice/user@1000.servic...",[1724748417093736128],b'',b'',0,1,"{'type': 'samples', 'unit': 'count', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSPEC...",True,
3,1724748423845,d1e535ee-0e87-477e-b31d-5253c589b97f,0,"[{'key': 'host.id', 'value': {'string_value': '0', 'bool_value': None, 'int_value': None, 'doubl...",0,,0,,,[],0,,0,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,[],0,,b'',,,1724748423643725554,0,"{'type': 'cpu', 'unit': 'nanoseconds', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSP...",50000000,[],,0,wPV4lODzFxdwY_wpGV2k5g,"[{'mapping': {'memory_start': 4194304, 'memory_limit': 28229632, 'file_offset': 0, 'filename': '...",[],"[{'key': 'container.id', 'value': {'string_value': '/user.slice/user-1000.slice/user@1000.servic...",[1724748423443695624],b'',b'',0,1,"{'type': 'samples', 'unit': 'count', 'aggregation_temporality': b'AGGREGATION_TEMPORALITY_UNSPEC...",True,
4,1724748423845,d1e535ee-0e87-477e-b31d-5253c589b97f,0,"[{'key': 'host.id', 'value': {'string_value': '0', 'bool_value': None, 'int_value': None, 'doubl...",0,,0,,,[],0,,0,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423

In [97]:
# extract meta-data
resource_attrs_df=raw_df[['resource_attributes']].apply(otel_attrs, axis=1).add_prefix('ra_')
scope_attrs_df=raw_df[['scope_attributes']].apply(otel_attrs, axis=1).add_prefix('sa_')
profile_attrs_df=raw_df[['profile_attributes']].apply(otel_attrs, axis=1).add_prefix('pa_')
attrs_df=raw_df[['attributes']].apply(otel_attrs, axis=1).add_prefix('a_')

In [98]:
%%markdown

#### Profiles resource meta-data

The resource attributes (or meta-data) is a lot simpler, as there's only one process generating `profiles` - the [OpenTelemetry eBPF profiler](https://github.com/open-telemetry/opentelemetry-ebpf-profiler).

At the time of writing, OpenTelemetry profiling is in an early stage of development and therefore is not built into the usual instrumentation agents and plugins. It runs as a separate process on the host operating system only.

For this reason, the `resource attributes` (or resource meta-data) are quite simple, specifying only the machine where the eBPF profiler process is running:


#### Profiles resource meta-data

The resource attributes (or meta-data) is a lot simpler, as there's only one process generating `profiles` - the [OpenTelemetry eBPF profiler](https://github.com/open-telemetry/opentelemetry-ebpf-profiler).

At the time of writing, OpenTelemetry profiling is in an early stage of development and therefore is not built into the usual instrumentation agents and plugins. It runs as a separate process on the host operating system only.

For this reason, the `resource attributes` (or resource meta-data) are quite simple, specifying only the machine where the eBPF profiler process is running:


In [99]:
resource_attrs_df.head(10).fillna('')

,ra_host.id,ra_host.ip,ra_host.name,ra_service.version,ra_os.kernel
0,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
1,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
2,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
3,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
4,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
5,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
6,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
7,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
8,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
9,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0


In [100]:
%%markdown

#### ...and sample meta-data

Also due to the early stage of development very few additional sample `attributes` are collected:


#### ...and sample meta-data

Also due to the early stage of development very few additional sample `attributes` are collected:


In [101]:
attrs_df.head(25)

,a_container.id,a_thread.name
0,/libpod_parent/libpod-0ba85205d552a1f876ba13ae5632576fc8e62f80b9a0f371c4c41ce233feac88/kubepods....,kube-apiserver
1,/user.slice/user-1000.slice/user@1000.service/app.slice/app-dbus\x2d:1.2\x2dorg.gnome.Nautilus.s...,java
2,/user.slice/user-1000.slice/user@1000.service/session.slice/org.gnome.Shell@wayland.service,gnome-shell
3,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-7...,kubectl
4,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-4...,C2 CompilerThre
5,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-4...,C2 CompilerThre
6,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-4...,ForkJoinPool-4-
7,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-4...,C2 CompilerThre
8,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-7...,kubectl
9,/user.slice/user-1000.slice/user@1000.service/app.slice/app-org.gnome.Terminal.slice/vte-spawn-7...,kubectl


In [102]:
%%markdown

# Preprocessing profiles

As `profiles` data is a bit more complex - containing **stack frames** that were observed - let's first do some transformations to get it to a format that's easier to work with.

More about stack frames later, but first, we'll extract the base columns and a few nested values:


# Preprocessing profiles

As `profiles` data is a bit more complex - containing **stack frames** that were observed - let's first do some transformations to get it to a format that's easier to work with.

More about stack frames later, but first, we'll extract the base columns and a few nested values:


In [103]:
base_df=raw_df.loc[raw_df['is_valid'] == True][
    [
        'profile_id',
        'start_time_unix_nano',
        'end_time_unix_nano',
        'time_nanos',
        'duration_nanos',
        'period',
    ]
]
base_df['period_type.type']=raw_df.loc[raw_df['is_valid'] == True]['period_type'].apply(lambda s: s.get('type'))
base_df['period_type.unit']=raw_df.loc[raw_df['is_valid'] == True]['period_type'].apply(lambda s: s.get('unit'))
base_df['period_type.aggregation_temporality']=raw_df.loc[raw_df['is_valid'] == True]['period_type'].apply(lambda s: s.get('aggregation_temporality'))
base_df['type.type']=raw_df.loc[raw_df['is_valid'] == True]['type'].apply(lambda s: s.get('type'))
base_df['type.unit']=raw_df.loc[raw_df['is_valid'] == True]['type'].apply(lambda s: s.get('unit'))
base_df['type.aggregation_temporality']=raw_df.loc[raw_df['is_valid'] == True]['type'].apply(lambda s: s.get('aggregation_temporality'))

base_df['stacktrace_id']=raw_df.loc[raw_df['is_valid'] == True]['stacktrace_id']
base_df['timestamps_unix_nano']=raw_df.loc[raw_df['is_valid'] == True]['timestamps_unix_nano']
base_df['trace_id']=raw_df.loc[raw_df['is_valid'] == True]['trace_id']
base_df['span_id']=raw_df.loc[raw_df['is_valid'] == True]['span_id']
base_df['value']=raw_df.loc[raw_df['is_valid'] == True]['value']

In [104]:
base_df.head(25)

,profile_id,start_time_unix_nano,end_time_unix_nano,time_nanos,duration_nanos,period,period_type.type,period_type.unit,period_type.aggregation_temporality,type.type,type.unit,type.aggregation_temporality,stacktrace_id,timestamps_unix_nano,trace_id,span_id,value
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',iTnG0oMa50esbnFSMfnBIA,[1724748417243725237],b'',b'',1
1,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',WYqKLHYa3mJGRunbvCksdA,[1724748417943747682],b'',b'',1
2,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',3q0cPtu3MDx3nKTLoW1dYw,[1724748417093736128],b'',b'',1
3,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',wPV4lODzFxdwY_wpGV2k5g,[1724748423443695624],b'',b'',1
4,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',llfhpWeEYOTUhd0Rz1B1mQ,[1724748418343758562],b'',b'',1
5,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',_MxCXoFscAJB-uUwu5nGWw,[1724748418343747849],b'',b'',1
6,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',CO4C0zwEX7XzCwJPdEiJcw,[1724748418343696197],b'',b'',1
7,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',MvDMdczrfYQ0MIWWfD3Tgw,[1724748418643758739],b'',b'',1
8,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',Qqa52rATLXbcNs6f5NQcuA,[1724748423393725765],b'',b'',1
9,b'\xb4])1\xe1L\xb7\xde\xa0lY\x82T\x89\xe1/',1724748423643725554,1724748423643725554,1724748423643725554,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',zpmkVzYMovFtWRjfqgKlxw,[1724748423343716192],b'',b'',1


In [105]:
%%markdown

# Stack traces (or stack frames)

Now let's take a look at what a **stack trace** is. You can think of profiling as a way of looking (once in a while) at what a CPU is doing at that moment. It could be executing a method within your app's code 
(like - parsing a JSON request from a client), it could be running the code of system library, or could be inside the OS kernel - reading data from a network socket for example.

This bit of information alone won't be particularly useful, as it does not tell you what lead to this particular code to be executed by the CPU. There might be multiple parts within your app code that allocate memory and
even though you can see a CPU as it's performing this operation - you wouldn't know why, or what code led to it. To give you that extra knowledge a profiler will report the entire **stack trace**, with all methods, as they
invoked each other.

Let's get a stack trace and see what's inside. Few things to notice in the table below:
- A `stacktrace_id` is globally unique for that **stack trace**, meaning, if the same **stack trace** is seen more than once - the same id will be recorded, within the same, or different, `profile_id`.
- There is a timestamp when each **stack trace** was observed (in the `timestamps_unix_nano` array)
- `locations` holds the actual **stack trace**


# Stack traces (or stack frames)

Now let's take a look at what a **stack trace** is. You can think of profiling as a way of looking (once in a while) at what a CPU is doing at that moment. It could be executing a method within your app's code 
(like - parsing a JSON request from a client), it could be running the code of system library, or could be inside the OS kernel - reading data from a network socket for example.

This bit of information alone won't be particularly useful, as it does not tell you what lead to this particular code to be executed by the CPU. There might be multiple parts within your app code that allocate memory and
even though you can see a CPU as it's performing this operation - you wouldn't know why, or what code led to it. To give you that extra knowledge a profiler will report the entire **stack trace**, with all methods, as they
invoked each other.

Let's get a stack trace and see what's inside. Few things to notice in the table below:
- A `stacktrace_id` is globally unique for that **stack trace**, meaning, if the same **stack trace** is seen more than once - the same id will be recorded, within the same, or different, `profile_id`.
- There is a timestamp when each **stack trace** was observed (in the `timestamps_unix_nano` array)
- `locations` holds the actual **stack trace**


In [106]:
raw_df[raw_df['stacktrace_id'] == '1SaPndvY2H8vw9XiXIIzCw'][[
    'profile_id',
    'stacktrace_id',
    'timestamps_unix_nano',
    'locations'
]]

,profile_id,stacktrace_id,timestamps_unix_nano,locations
2459,"b'`\xc3\xaePk\x1a\x84\x0fe,\x89y\x0f\xc3\x90\xbd'",1SaPndvY2H8vw9XiXIIzCw,[1724748441393709769],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
3330,b'\xc5\xee\x88\x8c\xfer\x1b\x95\x11\xb3\x96\xf4\x80vKS',1SaPndvY2H8vw9XiXIIzCw,[1724748457943748631],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
3340,b'\xc5\xee\x88\x8c\xfer\x1b\x95\x11\xb3\x96\xf4\x80vKS',1SaPndvY2H8vw9XiXIIzCw,"[1724748457943716145, 1724748458543728744]","[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
4701,b'W\xbb\xce\xec\xf6\xb8yD\xde\xf9G?O\x1b\xa8L',1SaPndvY2H8vw9XiXIIzCw,[1724748523843736513],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
6193,b'\x17%5\xa0\xe6\xdd\xbc\xc9\xe0\xe6\xff\xb9e\xac\xfe\xff',1SaPndvY2H8vw9XiXIIzCw,[1724748598443748074],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
6227,b'\x17%5\xa0\xe6\xdd\xbc\xc9\xe0\xe6\xff\xb9e\xac\xfe\xff',1SaPndvY2H8vw9XiXIIzCw,[1724748598443736912],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
6378,b'\x85\xcapHh\x1e8\x81d\xe3c>v\xd5\xd5p',1SaPndvY2H8vw9XiXIIzCw,[1724748604243742965],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
6407,b'\x85\xcapHh\x1e8\x81d\xe3c>v\xd5\xd5p',1SaPndvY2H8vw9XiXIIzCw,[1724748604243759316],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
6879,"b""\\\x1b\xb0V\x0e'm\x1d\xb9-B\x92|\xfd\x96\xb6""",1SaPndvY2H8vw9XiXIIzCw,[1724748609193743007],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."
7888,b'\xd2\x08W\xb3\xdc\xb4O8\x02\x02\xa14\xd8f\xd9\xa3',1SaPndvY2H8vw9XiXIIzCw,[1724748635493727029],"[{'mapping': {'memory_start': 0, 'memory_limit': 20873216, 'file_offset': 0, 'filename': 'libjvm..."


In [107]:
%%markdown

# Extract the stack frames

Now, let's also get each entry of each stack frame, in a separate DataFrame.

Note how these may point to addresses, functions, line numbers in files (`location.type` column):


# Extract the stack frames

Now, let's also get each entry of each stack frame, in a separate DataFrame.

Note how these may point to addresses, functions, line numbers in files (`location.type` column):


In [108]:
# define a helper function to extract some locations data:
def otel_profile_location(series):
    location = series.iloc[0]
    mapping = location['mapping']
    if location['lines'].size > 0:
        line_0 = location['lines'][0]
        function_line_0 = line_0['function']
        line_line_0 = line_0['line']
        column_line_0 = line_0['column']
    else:
        function_line_0 = {}
        line_line_0 = None
        column_line_0 = None
    
    address = location['address']
    is_folded = location['is_folded']
    typ = location['type']
    attrs = location['attributes']
    keys = [
        *[f'location.map.{mk}' for mk in mapping.keys()],
        'location.address',
        *[f'location.line_0.function.{fk}' for fk in function_line_0.keys()],
        'location.line_0.line',
        'location.line_0.column',
        'location.is_folded',
        'location.type',
        'location.attributes'
    ]
    values = [
        *mapping.values(),
        address,
        *function_line_0.values(),
        line_line_0,
        column_line_0,
        is_folded,
        typ,
        attrs
    ]
    return pd.Series(values, index=keys)

# explode each location into a row of its own:
locations_df=raw_df[['stacktrace_id']].join(raw_df[['locations']].explode('locations').apply(otel_profile_location, axis=1))

In [109]:
locations_df[locations_df['stacktrace_id'] == 'UuAAmyCUEGdKfbpdRN9-dA'][[
    'stacktrace_id', 'location.address', 'location.line_0.function.filename', 'location.line_0.function.name', 'location.line_0.line',
    'location.map.file_offset', 'location.map.filename', 'location.type'
]].fillna('')

,stacktrace_id,location.address,location.line_0.function.filename,location.line_0.function.name,location.line_0.line,location.map.file_offset,location.map.filename,location.type
9589,UuAAmyCUEGdKfbpdRN9-dA,3456007,,shrink_folio_list,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3459214,,evict_folios,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3463669,,try_to_shrink_lruvec,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3471569,,shrink_lruvec,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3472884,,shrink_node_memcgs,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3478330,,shrink_node,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3482359,,shrink_zones.constprop.0,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3483025,,do_try_to_free_pages,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,3485780,,try_to_free_mem_cgroup_pages,0.0,0,,kernel
9589,UuAAmyCUEGdKfbpdRN9-dA,4222703,,try_charge_memcg,0.0,0,,kernel


In [110]:
%%markdown

# gRPC sends a message (stack trace)

The table above shows a single **stack trace** sample, where the gRPC java implementation (using Netty) invoked an OS **syscall**, which in turn caused Linux to send a message to a gRPC client.

With stack traces you can track the performance of each method in your code, that is - at any depth in the stack. For example - you can investigate what has the most immediate effect of your method's performance by taking all stack frames with it
and looking at what's below (at lower depth) your method.

> ***Important:***
>
> ***Stack traces are sampled.***
>
> The OpenTelemetry eBPF profiler will ***not*** send data about everything CPUs have done.


# gRPC sends a message (stack trace)

The table above shows a single **stack trace** sample, where the gRPC java implementation (using Netty) invoked an OS **syscall**, which in turn caused Linux to send a message to a gRPC client.

With stack traces you can track the performance of each method in your code, that is - at any depth in the stack. For example - you can investigate what has the most immediate effect of your method's performance by taking all stack frames with it
and looking at what's below (at lower depth) your method.

> ***Important:***
>
> ***Stack traces are sampled.***
>
> The OpenTelemetry eBPF profiler will ***not*** send data about everything CPUs have done.


In [111]:
%%markdown

# So what is a Profile then?

An OpenTelemetry `profile` works as a collection of **stack trace samples,** observed over a period (`start_time_unix_nano` to `end_time_unix_nano`).

Here's a `profile` and it's contained `stacktrace_id`s and the `timestamps_unix_nano` when each of them was observed.

Note that within a `profile` (that is - within an observational period) a `stacktrace_id` might be more than once, and there will be two or more `timestamps_unix_nano` next to it:


# So what is a Profile then?

An OpenTelemetry `profile` works as a collection of **stack trace samples,** observed over a period (`start_time_unix_nano` to `end_time_unix_nano`).

Here's a `profile` and it's contained `stacktrace_id`s and the `timestamps_unix_nano` when each of them was observed.

Note that within a `profile` (that is - within an observational period) a `stacktrace_id` might be more than once, and there will be two or more `timestamps_unix_nano` next to it:


In [112]:
base_df[base_df['profile_id'] == b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|']

,profile_id,start_time_unix_nano,end_time_unix_nano,time_nanos,duration_nanos,period,period_type.type,period_type.unit,period_type.aggregation_temporality,type.type,type.unit,type.aggregation_temporality,stacktrace_id,timestamps_unix_nano,trace_id,span_id,value
10115,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',blmYb1bfOlUKagj7aubCaw,[1724748728843748170],b'',b'',1
10116,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',aW3pt3778x7zYVf7ofqMLQ,[1724748727043716384],b'',b'',1
10117,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',tAefttlPgCyFRVEBovTjtA,[1724748727643696101],b'',b'',1
10118,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',B001bWhXisiPxwHZPeEmqw,[1724748729893742215],b'',b'',1
10119,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',Rvrus5PWAvvrgv5rUn4Y2A,[1724748727393747793],b'',b'',1
10120,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',Sy74jS3nIV5unXn3b-Minw,[1724748729143725526],b'',b'',1
10121,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',TsKGOrUN_KS9EG1g0ZUQwg,[1724748728293716293],b'',b'',1
10122,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',xSWPjcBs0w_PjB-i_tW3IQ,[1724748729143747831],b'',b'',1
10123,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',DjJDbyRs7c5_nw-IQ60Zlw,[1724748727643742056],b'',b'',1
10124,b'\x929\x10\x9cy&\xc9v\x8a\xae\xeb\xe8\x06\x1c\xa1|',1724748727643709572,1724748727643714323,1724748727643709572,4751,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',fZ0kJ0QqXjC4oIcwx3I00w,[1724748728843709654],b'',b'',1


In [113]:
%%markdown

#### Join the DataFrames

To complete the pre-processing lets join the base data frame with the **stack frames** and all meta-data. We'll also precompute a few extra columns: `stacktrace_len` - equal to the total number of entries in a stack frame,
`stacktrace_max_depth` - equal to the maximum depth, and `stacktrace_depth` - the depth of the current entry into its containing stack frame: 


#### Join the DataFrames

To complete the pre-processing lets join the base data frame with the **stack frames** and all meta-data. We'll also precompute a few extra columns: `stacktrace_len` - equal to the total number of entries in a stack frame,
`stacktrace_max_depth` - equal to the maximum depth, and `stacktrace_depth` - the depth of the current entry into its containing stack frame: 


In [114]:
ext_df=base_df.join(locations_df.drop(['stacktrace_id'], axis=1)).join(
    locations_df[['location.address']].groupby(level=0).count().rename({'location.address': 'stacktrace_len'}, axis=1)
)
ext_df['stacktrace_max_depth'] = ext_df['stacktrace_len'] - 1
ext_df['stacktrace_depth'] = ext_df['stacktrace_len'] - locations_df[['location.address']].groupby(level=0).transform('cumcount') - 1
ext_df=ext_df.join(attrs_df).join(profile_attrs_df).join(resource_attrs_df).join(scope_attrs_df)
ext_df.head(25).fillna('')

,profile_id,start_time_unix_nano,end_time_unix_nano,time_nanos,duration_nanos,period,period_type.type,period_type.unit,period_type.aggregation_temporality,type.type,type.unit,type.aggregation_temporality,stacktrace_id,timestamps_unix_nano,trace_id,span_id,value,location.address,location.attributes,location.is_folded,location.line_0.column,location.line_0.function.filename,location.line_0.function.name,location.line_0.function.start_line,location.line_0.function.system_name,location.line_0.line,location.map.attributes,location.map.build_id,location.map.build_id_kind,location.map.file_offset,location.map.filename,location.map.has_filenames,location.map.has_functions,location.map.has_inline_frames,location.map.has_line_numbers,location.map.memory_limit,location.map.memory_start,location.type,stacktrace_len,stacktrace_max_depth,stacktrace_depth,a_container.id,a_thread.name,ra_host.id,ra_host.ip,ra_host.name,ra_service.version,ra_os.kernel
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',iTnG0oMa50esbnFSMfnBIA,[1724748417243725237],b'',b'',1,1865057,[],False,0.0,,hrtimer_start_range_ns,0.0,,0.0,[],01e5f43a8d64184c0d3729421072863b,b'BUILD_ID_BINARY_HASH',0,,False,False,False,False,0,0,kernel,15,14,14,/libpod_parent/libpod-0ba85205d552a1f876ba13ae5632576fc8e62f80b9a0f371c4c41ce233feac88/kubepods....,kube-apiserver,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',iTnG0oMa50esbnFSMfnBIA,[1724748417243725237],b'',b'',1,13197095,[],False,0.0,,schedule_hrtimeout_range_clock,0.0,,0.0,[],01e5f43a8d64184c0d3729421072863b,b'BUILD_ID_BINARY_HASH',0,,False,False,False,False,0,0,kernel,15,14,13,/libpod_parent/libpod-0ba85205d552a1f876ba13ae5632576fc8e62f80b9a0f371c4c41ce233feac88/kubepods....,kube-apiserver,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',iTnG0oMa50esbnFSMfnBIA,[1724748417243725237],b'',b'',1,4797559,[],False,0.0,,ep_poll,0.0,,0.0,[],01e5f43a8d64184c0d3729421072863b,b'BUILD_ID_BINARY_HASH',0,,False,False,False,False,0,0,kernel,15,14,12,/libpod_parent/libpod-0ba85205d552a1f876ba13ae5632576fc8e62f80b9a0f371c4c41ce233feac88/kubepods....,kube-apiserver,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',iTnG0oMa50esbnFSMfnBIA,[1724748417243725237],b'',b'',1,4797856,[],False,0.0,,do_epoll_wait,0.0,,0.0,[],01e5f43a8d64184c0d3729421072863b,b'BUILD_ID_BINARY_HASH',0,,False,False,False,False,0,0,kernel,15,14,11,/libpod_parent/libpod-0ba85205d552a1f876ba13ae5632576fc8e62f80b9a0f371c4c41ce233feac88/kubepods....,kube-apiserver,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe3\x8e\xe1\xa2',1724748417093736128,1724748417093736128,1724748417093736128,0,50000000,cpu,nanoseconds,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',samples,count,b'AGGREGATION_TEMPORALITY_UNSPECIFIED',iTnG0oMa50esbnFSMfnBIA,[1724748417243725237],b'',b'',1,4798187,[],False,0.0,,do_epoll_pwait.part.0,0.0,,0.0,[],01e5f43a8d64184c0d3729421072863b,b'BUILD_ID_BINARY_HASH',0,,False,False,False,False,0,0,kernel,15,14,10,/libpod_parent/libpod-0ba85205d552a1f876ba13ae5632576fc8e62f80b9a0f371c4c41ce233feac88/kubepods....,kube-apiserver,0,127.0.0.1,rh-ho-01.mishmash.local,,5.14.0
0,b'Xp\xa4\x95\xcbR?x\xa6Y\x19\x80\xe

In [115]:
%%markdown

## Save the pre-processed profiles to Parquet:


## Save the pre-processed profiles to Parquet:


In [116]:
ext_df.convert_dtypes().to_parquet('otel-demo-app/profiles_experimental.parquet', engine='pyarrow')